In [12]:
import numpy as np
import pandas as pd

%matplotlib osx
import matplotlib.pyplot as plt
import matplotlib as mp

# from m_gncspline import global_natural_spline
# from gcvspline import GCVSmoothedNSpline, SmoothedNSpline

import glob
from scipy.signal import find_peaks
from matplotlib.lines import Line2D
import cmasher as cmr

import os
cwd = os.getcwd()

# Load Data

## <i>Chrysopelea</i>

In [4]:
#import chrysopelea data
import dill

with open('smooth_pos_v8.pkl', 'rb') as var:
    sm_pos = dill.load(var)
    
with open('smooth_vel_v8.pkl', 'rb') as var:
    sm_vel = dill.load(var)
    
with open('raw_dictionaries_v9.pkl', 'rb') as var:
    all_imported = dill.load(var)
    
cp_df = pd.read_csv(cwd+'/Chrysopelea NC Data.csv')
cp_df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'smooth_pos_v8.pkl'

In [3]:
#create list of reference dictionaries and trials that correspond to those in the dataframe (NCs only)
cp_trials = [each['tn'] for each in all_imported]
NC_inds = [cp_trials.index(n) for n in list(cp_df['tn'].values)]

NC_data = [all_imported[n] for n in NC_inds]
NC_trials = [sm_pos[n] for n in NC_inds]
NC_vels = [sm_vel[n] for n in NC_inds]

NC_data[23]['fn'] = np.arange(41107,41660) #this trial has a cropping issue.

In [4]:
#make sure list and dataframe have trials in the same order
list1 = [each['tn'] for each in NC_data]
diffs = [i-j for i,j in zip(list(cp_df['tn'].values),list1)]
print(min(diffs),max(diffs))

0 0


In [5]:
## Baseline lists: gap size presented in centimeters and svl
cp_gsr = [each['gs_%'] for each in NC_data]
cp_gscm = [each['gs_m']*100 for each in NC_data]

## <i>Dendrelaphis</i>

In [6]:
#import Dendrelaphis data
dp_df = pd.read_csv(cwd+'/Dendrelaphis NC Data.csv')
dp_df.dropna(axis=0,how="all",inplace=True)
dp_df.dropna(axis=1,how="all",inplace=True)
dp_df.head()

#create list of all head position data
pfiles = glob.glob(cwd+'/Processed Files/*')
pfiles.sort()
position = [pfiles[i] for i in np.arange(len(pfiles)) if pfiles[i][-12:-4]=='smoothed']

#create list of all head velocity data
velocity = [pfiles[i] for i in np.arange(len(pfiles)) if pfiles[i][-12:-4]=='velocity']
dp_vel = [pd.read_csv(item,header=None) for item in velocity]

In [7]:
# create lists of gap size in % SVL and cm
df_list = list(dp_df['tn'])
dp_gsr = []
dp_gscm = []

for i in np.arange(len(position)):
    trial = int(position[i][-25:-22]) #trial number from the file
    ind = df_list.index(trial)
    
    svl = dp_df['svl'][ind]/100 #cm to m    
    
    origin = np.array([dp_df['orig_x'][ind],dp_df['orig_y'][ind],dp_df['orig_z'][ind]])
    target = np.array([dp_df['targ_x'][ind],dp_df['targ_y'][ind],dp_df['targ_z'][ind]])
    
    gs = np.linalg.norm(target-origin) #in meters
    gscm = gs*100
    gsr = gs/svl*100 #in percent svl terms
    dp_gsr.append(gsr)
    dp_gscm.append(gscm)
    

# Generate Data Table

## Metrics 1 & 2: Horizontal & Vertical variation (head)

### <i>Chrysopelea</i>

In [20]:
cp_hvar = []
cp_zvar = []
cp_hvar_tf = []
cp_zvar_tf = []

cp_heads = [item[:,0,:] for item in NC_trials] #a list of head position data, recorded in mm -- HAS NANs

for ind in np.arange(len(cp_heads)):

    snake = cp_heads[ind]/1000.0 #change from mm to m
    tf = cp_df['transition'][ind] #transition frame
    start = cp_df['start'][ind]
    stop = cp_df['stop'][ind]
    
    
    frames = list(NC_data[ind]['fn']) #for translating frame into data index 
    start_i = frames.index(start)
    stop_i = frames.index(stop)
    
    hvar = np.nanmax(snake[start_i:stop_i,1])-np.nanmin(snake[start_i:stop_i,1]) #horizontal variation, y.
    zvar = np.nanmax(snake[start_i:stop_i,2])-np.nanmin(snake[start_i:stop_i,2]) #vertical variation, z.
    
    cp_hvar.append(hvar)
    cp_zvar.append(zvar)

    if not np.isnan(tf):
        tf = int(tf)
        tf_i = frames.index(tf) 
    
        #create variation after TF
        hvar_tf = np.nanmax(snake[tf_i:stop_i,1])-np.nanmin(snake[tf_i:stop_i,1]) #horizontal variation after tf, y.
        zvar_tf = np.nanmax(snake[tf_i:stop_i,2])-np.nanmin(snake[tf_i:stop_i,2]) #vertical variation after tf, z.
        
        cp_hvar_tf.append(hvar_tf)
        cp_zvar_tf.append(zvar_tf)

        
    else:
        cp_hvar_tf.append(np.nan)
        cp_zvar_tf.append(np.nan)

### <i>Dendrelaphis</i>

In [21]:
dp_hvar = []
dp_zvar = []
dp_hvar_tf = []
dp_zvar_tf = []

for i in np.arange(len(position)):
    trial = int(position[i][-25:-22]) #trial number from the file
    ind = df_list.index(trial)
    
    snake = np.array(pd.read_csv(position[i]))
    
    #overall variation
    hvar = max(snake[:,1])-min(snake[:,1]) #horizontal variation, y.
    zvar = max(snake[:,2])-min(snake[:,2])#vertical variation, z.
    
    dp_hvar.append(hvar)
    dp_zvar.append(zvar)
    
    #variation after transition
    tf = dp_df['transition'][ind]
    start = int(dp_df['start'][ind])
    
    if not np.isnan(tf):
        tf = int(tf-start) #transition frame relative to start frame gives index.
    
        #create variation after TF
        hvar_tf = max(snake[tf:,1])-min(snake[tf:,1])
        zvar_tf = max(snake[tf:,2])-min(snake[tf:,2])
        
        dp_hvar_tf.append(hvar_tf)
        dp_zvar_tf.append(zvar_tf)
        
    else:
        dp_hvar_tf.append(np.nan)
        dp_zvar_tf.append(np.nan)        

## Metrics 3-5: Max, landing, and average head speed

### <i>Chrysopelea</i>

In [22]:
cp_vd = pd.read_csv(cwd+'/cp_NC_vdata.csv')
cp_maxv = list(cp_vd['mhv'].values)
cp_forward = list(cp_vd['axv'].values)
cp_landv = list(cp_vd['landv'].values)

#do these later if it seems interesting.
cp_maxv_tf = []
cp_forward_tf = []

for i in np.arange(len(cp_maxv)):
    
    cp_maxv_tf.append(np.nan)
    cp_forward_tf.append(np.nan)

### <i>Dendrelaphis</i>

In [23]:
#velocity metrics for whole trial,and after transition (to remove approach phase).

dp_maxv = []
dp_forward = []
dp_landv = []

dp_maxv_tf = []
dp_forward_tf = []

for i in np.arange(len(dp_vel)):
    
    trial = int(velocity[i][-25:-22]) #trial number from the file
    ind = df_list.index(trial)
    
    dp_headv = dp_vel[i]
    resultant = np.linalg.norm(dp_headv,axis=1) #calculate resultant velocity
    
    land_speed = resultant[-1] #final frame resultant velocity = landing frame
    max_speed = np.max(resultant[:]) #max overall  
    X = list(resultant)
    forward = [v for v in X if v > 0.02] #looked a resultant graph in region where snake does not seem to be moving.
    avg_speed = np.mean(forward)
    
    dp_maxv.append(max_speed)
    dp_forward.append(avg_speed)
    dp_landv.append(land_speed)
    
    #speeds after transition
    tf = dp_df['transition'][ind]
    start = int(dp_df['start'][ind])
    
    if not np.isnan(tf):
        tf = int(tf-start) #transition frame relative to start frame gives index.
        
        max_tf = np.max(resultant[tf:]) #max during lunge event (i.e. from acceleration frame onwards)
        X1 = list(resultant[tf:])
        forward1 = [v for v in X1 if v > 0.02]
        avg_speed1 = np.mean(forward1)
        
        dp_maxv_tf.append(max_tf)
        dp_forward_tf.append(avg_speed1) 
        
    else:
        dp_maxv_tf.append(np.nan)
        dp_forward_tf.append(np.nan)         

## Metrics 6-8: Overshoot, distance traveled from AF, and Zpos at AF, Xpos at AF.

### <i>Chrysopelea</i>

In [8]:
cp_over = []
cp_dtaf = []
cp_zpaf = []
cp_zpmx = []
cp_dft = []

cp_heads = [item[:,0,:] for item in NC_trials] #a list of head position data, recorded in mm -- HAS NANs

for ind in np.arange(len(cp_heads)):
    trial = NC_data[ind]['tn']
    svl = NC_data[ind]['svl']/100 #cm to m    

    snake = cp_heads[ind]/1000.0 #change from mm to m
    acc = cp_df['acceleration'][ind] #acceleration frame
    start = cp_df['start'][ind]
    stop = cp_df['stop'][ind]
    
    if trial in [85,93,94]:
        origin = snake[0] #origin marker was missing for these trials; use head position at start as substitute
        
    else:
        origin = [0,0,0] #origin position
    
    target = NC_data[ind]['nte']/1000.0 #change from mm to m
    
    frames = list(NC_data[ind]['fn']) #for translating frame into data index 
    start_i = frames.index(start)
    stop_i = frames.index(stop)
    
    over = snake[-1,0] - target[0] #x distance overshooting the target distance
    cp_over.append(over) #meters; overshoot

    if not np.isnan(acc):
        acc = int(acc)
        acc_i = frames.index(acc) 

        distance = np.linalg.norm(snake[-1,:]-origin) #total distance traveled, relative to origin
        zpos = snake[acc_i,2]-origin[2] #z position at acceleration start, relative to level of branches
        zpos_max = np.nanmax(snake[acc_i:,2])-origin[2] #z position at max (after AF) relative to level of branches
        dft_af = np.linalg.norm(target-snake[acc_i,:])
        
        cp_zpaf.append(zpos) #meters; z position at acceleration frame
        cp_dtaf.append(distance) #meters; distance traveled from acceleration frame to landing
        cp_zpmx.append(zpos_max)
        cp_dft.append(dft_af) #meters; distance from target at acceleration
        
    else:
        cp_zpaf.append(np.nan) 
        cp_dtaf.append(np.nan) 
        cp_dft.append(np.nan)
        cp_zpmx.append(np.nan)

### <i>Dendrelaphis</i>

In [9]:
dp_over = []
dp_dtaf = []
dp_zpaf = []
dp_zpmx = []
dp_dft = []

for i in np.arange(len(position)):
    trial = int(position[i][-25:-22]) #trial number from the file
    ind = df_list.index(trial)
    
    svl = dp_df['svl'][ind]/100 #cm to m    
    snake = np.array(pd.read_csv(position[i]))
    
    af = dp_df['acceleration'][ind] - dp_df['start'][ind]#acceleration frame relative to start fraame
    
    origin = [dp_df['orig_x'][ind],dp_df['orig_y'][ind],dp_df['orig_z'][ind]]
    target = [dp_df['targ_x'][ind],dp_df['targ_y'][ind],dp_df['targ_z'][ind]]
    
    over = snake[-1,0] - target[0] #x distance past target end at landing
    zpos = (snake[af,2]-origin[2]) #z distance from branch at acceleration start
    zpos_max = max(snake[af:,2])-origin[2] #zmax after AF from branches
    distance = np.linalg.norm(snake[-1,:]-origin) #total distance traveled, relative to origin
    dft_af = np.linalg.norm(target - snake[af,:]) #total distance from target at AF

    dp_over.append(over)
    dp_dtaf.append(distance)
    dp_zpaf.append(zpos)
    dp_zpmx.append(zpos_max)
    dp_dft.append(dft_af)

## Metrics 9-11: AH and LD at AF + HP

In [10]:
def snake_shape1(n_snake, n_sp, svl, orig_x, orig_z): 
    """takes a single frame of snake marker positions and a list of marker spacings
    Returns arc height and loop depth.
    
    inputs:
    snake = m markers by 3 dimensions file (meters)
    marks = list of m-1 marker spacings; distance from first marker to second marker is first entry (meters)
    svl = snake snout vent length (meters)
    orig_x = X dimensions of origin position 
    orig_z = Z dimension of origin position
    
    outputs:
    AH = distance from head to the lowest point.
    LD = distance from origin end to the lowest body point.
    relative versions are in terms of %svl, regular are in meters.
    """
    ### 1: Make the spline
    out = global_natural_spline(n_snake,np.array(n_sp),1000) #fit a spline of 1000 points between the known markers positions
    r, dr, ddr, dddr, ts, ss, seg_lens, lengths_total, idx_pts = out    

    ###2: Calculate AH and LD using the spline
    
    #crop spline to section after the end of the origin
    # find where X > X_origin (the body is in the gap)
    
    idx_gap = np.where(r[:, 0] > orig_x)[0]
    r_gap = r[idx_gap]

    head_z = r_gap[0,2]
    low_z = r_gap[1:,2].min()

    AH = head_z-low_z
    LD = orig_z-low_z

    rel_AH = AH/svl*100
    rel_LD = LD/svl*100

    
    return AH, LD, rel_AH, rel_LD

### <i>Chrysopelea</i>

In [11]:
def fix_single_frame(SP_snake, spaces):
    """
    NOTE: must be at least 2 markers present in the frame.
    purpose: corrects an array of snake position data from a single frame, 
    and corresponding list of marker spacings, for markers that are missing 
    because they weren't visible in that frame
    
    snake = n x 3 array containing marker position data in 3 dimensions for n markers, measured in meters
    mark_spaces = list of n-1 spacings in meters between n markers
    """

    del_list = []
    n_snake = np.copy(SP_snake)
    n_sp = np.array(spaces)

    for t in np.arange(len(n_sp)+1):
        mark = n_snake[t,0]
        if np.isnan(mark): #make a list of markers to delete - delete ones with nans
            del_list.append(t)

    for l in reversed(del_list):
        n_snake=np.delete(n_snake,l,0) #delete the rows in the snake array
        if l==len(n_sp): #if the index corresponds to the last marker in the spacing array, just delete it
            n_sp=np.delete(n_sp,l-1)
        else:
            n_sp[l]=n_sp[l-1]+n_sp[l] #if the index corresponds to some other marker, get the appropriate spacing
            n_sp=np.delete(n_sp,l-1) #and then delete.

    
    return n_snake, n_sp

In [12]:
cp_LDs = []
cp_AHs = []

for i in np.arange(len(NC_trials)):
    svl = cp_df['svl'][i]/100.0 #cm to meters
    af = cp_df['acceleration'][i] #acceleration frame
    frames = list(NC_data[i]['fn']) #for translating frame into data index 
    af_i = frames.index(af) 
    
    sdata =  NC_trials[i]/1000.0 #mm to meters     
    trial = NC_data[i]['tn']
    if trial in [85,93,94]:
        origin = sdata[0,0,:] #origin marker was missing for these trials; use head position at start as substitute

    else:
        origin = [0,0,0] #origin position
    
    snake = sdata[af_i]
    
    if np.isnan(snake).any():
        marks = NC_data[i]['cms'][1:]
        n_snake,n_sp = fix_single_frame(snake,marks)
        AH, LD, rel_AH, rel_LD = snake_shape1(n_snake, np.array(n_sp), svl, origin[0], origin[2])
        cp_LDs.append(LD)
        cp_AHs.append(AH)
              
    else:
        marks = NC_data[i]['cms'][1:] #skip first entry nosetip to first marker spacing.
        marks = [t/100 for t in marks]
            
        AH, LD, rel_AH, rel_LD = snake_shape1(snake, marks, svl, origin[0], origin[2])
        cp_LDs.append(LD)
        cp_AHs.append(AH)

### <i>Dendrelaphis</i>

#### Import and align AF all-marker data

In [13]:
def get_rotation_matrix(i_v, unit=None):
    # From http://www.j3d.org/matrix_faq/matrfaq_latest.html#Q38
    if unit is None:
        unit = [1.0, 0.0, 0.0]
    # Normalize vector length
    i_v /= np.linalg.norm(i_v)

    # Get axis
    uvw = np.cross(unit,i_v)

    # compute trig values - no need to go through arccos and back
    rcos = np.dot(i_v, unit)
    rsin = np.linalg.norm(uvw)

    #normalize and unpack axis
    if not np.isclose(rsin, 0):
        uvw /= rsin
    u, v, w = uvw

    # Compute rotation matrix - re-expressed to show structure
    return (
        rcos * np.eye(3) +
        rsin * np.array([
            [ 0, -w,  v],
            [ w,  0, -u],
            [-v,  u,  0]
        ]) +
        (1.0 - rcos) * uvw[:,None] * uvw[None,:]
    )

def fix_coords(stuff,origin,target,gravity):
    """stuff: a nframes x 3 dims numpy array with data to be re-aligned to a new coordinate system where Z aligns with gravity
    origin: a 3d vector specifying the oriigin of the coordinate system
    target: a 3d vector specifying the x axis of the coordinate system (points from origin to target)
    gravity: a 3D unit vector that specifies the direction of gravity (down) in the original coordinate system"""
    
    
    rmat = get_rotation_matrix(gnorm, unit=[0.0,0.0,-1.0])

    oe = np.dot(origin, rmat)
    te = np.dot(target, rmat)

    stuff1 = np.copy(stuff)
    
    for i in np.arange(len(stuff)):
        stuff1[i,:] = np.dot(stuff[i,:].T, rmat)
        
    
    X = np.array([te[0,0] - oe[0,0], te[0,1]-oe[0,1],0])
    rmat1 = get_rotation_matrix(X ,unit=[1,0,0])
    
    stuff2 = np.copy(stuff)
    for i in np.arange(len(stuff)):
        stuff2[i,:] = np.dot(stuff1[i,:].T, rmat1)
    
    return stuff2

In [14]:
kfiles = glob.glob(cwd+'/Key Frames CSVs/*')
kfiles.sort()

key = pd.read_csv(cwd+'/DP_Posture_Reference.csv')
key.dropna(axis=0,how="all",inplace=True)
key.dropna(axis=1,how="all",inplace=True)
key.head()

,tn,ID,svl,start,acceleration,reference,orig_x,targ_x,orig_y,targ_y,...,space7,space8,space9,space10,space11,space12,space13,space14,space15,space16
0,9,2018DP01,113.900000,1,1401,1498,-0.278842,0.268839,0.153241,0.153241,...,10.37,7.96,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10,2018DP01,113.900000,1,865,1,-0.280206,0.213427,0.150346,0.150346,...,10.37,7.96,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,11,2018DP01,113.900000,1,365,1,-0.306503,0.211106,0.143803,0.143803,...,10.37,7.96,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,14,2018DP01,113.900000,1,1494,847,-0.232372,0.315085,0.152490,0.152490,...,10.37,7.96,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,42,2018DP02,127.443333,8063,8712,4468,-0.342431,0.202844,-0.399090,-0.399090,...,8.33,10.02,9.89,9.1,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
kfile_trials = [int(kfiles[n][-29:-26]) for n in np.arange(len(kfiles))]
key_trials = list(key['tn'].values)
check = [i-j for i,j in zip(key_trials,kfile_trials)]
print(max(check))
print(min(check))

0
0


In [16]:
#adjust position data to correct coordinate system
dims = ['X','Y','Z']
vals = ['pt2_','pt3_','pt4_','pt5_']
drops = [t+i for t in vals for i in dims] #used to remove non-snake columns later

postures = []

for n in np.arange(len(kfiles)):
    
    data = pd.read_csv(kfiles[n])
    frame = key.loc[n]['reference']
    trial = key.loc[n]['tn']
      
    #pull out relevant points: gravity, origin and target ends. 
    glow = drops[6:9]    
    ghigh = drops[9:12]
    
    g1 = data[glow].loc[frame-1].values
    g2 = data[ghigh].loc[frame-1].values
    grav = g1-g2
    gnorm = grav/np.linalg.norm(grav)
    
    OE = np.array(data.loc[[frame-1],['pt2_X','pt2_Y','pt2_Z']])
    TE = np.array(data.loc[[frame-1],['pt3_X','pt3_Y','pt3_Z']])
        
    #nan check #1
    if np.isnan(np.sum(g2)):
        print(str(trial)+": ghigh has nans")
        continue
        
    #nan check # 2
    if np.isnan(np.sum(g1)):
        print(str(trial)+": glow has nans")
        continue
    
    #nan check 3
    if np.isnan(np.sum(OE)):
        print(str(trial)+": OE has nans")
        continue
        
    #nan check 4
    if np.isnan(np.sum(TE)):
        print(str(trial)+": glow has nans")
        continue 
        
    OE = OE.reshape(1,3)
    TE = TE.reshape(1,3)
    g1 = g1.reshape(1,3)
    g2 = g2.reshape(1,3)
 
    #get snake position for accelerations frame
    acc_frame = key['acceleration'][n]-1
    
    #get rid of non-snake columns
    snake = data.drop(drops, axis=1)

    #get the marker positions of the snake at the acceleration frame
    posits = snake.iloc[acc_frame]
    posits = posits.dropna() #get rid of missing data (NaNs)

    #transpose to work with alignment function and change to array
    posits = np.array(posits).reshape(-1,3)

    #align coordinates
    oe1 = fix_coords(OE,OE,TE,gnorm)
    te1 = fix_coords(TE,OE,TE,gnorm)
    g1a = fix_coords(g1,OE,TE,gnorm)
    g2a = fix_coords(g2,OE,TE,gnorm)
    posit1 = fix_coords(posits,OE,TE,gnorm)
    
    postures.append(posit1) #save a list of all the re-aligned acceleration frame marker positions 

#### Save a graph of each spline

In [25]:
all_spacing_columns = key.columns[-16:-1] #use to get spacing columns later
all_spacing_columns

Index(['space1', 'space2', 'space3', 'space4', 'space5', 'space6', 'space7',
       'space8', 'space9', 'space10', 'space11', 'space12', 'space13',
       'space14', 'space15'],
      dtype='object')

In [39]:
for n in np.arange(len(kfiles)):
    trial = key.loc[n]['tn']
    n_snake = postures[n]
        
    orig_z = key['orig_z'][n] 
    orig_x = key['orig_x'][n]
    
    spacing_columns = all_spacing_columns[:len(n_snake)-1]        
    spacings = list(key[spacing_columns].loc[n])        
    n_sp = [x/100 for x in spacings if np.isnan(x) == False] #change cm to m and drop nans
    
    
    ### 1: Make the spline
    out = global_natural_spline(n_snake,np.array(n_sp),1000) #fit a spline of 1000 points between the known markers positions
    r, dr, ddr, dddr, ts, ss, seg_lens, lengths_total, idx_pts = out
    
    fig, ax = plt.subplots()
    ax.plot(r[:,0],r[:,2])
    ax.scatter(n_snake[:,0],n_snake[:,2])
    ax.scatter(orig_x,orig_z)
    ax.set_aspect('equal')
    
    
    plt.savefig(cwd+"/Graphs/AF Splines/"+str(trial)+'_AFSpline.pdf')
    plt.close()


In [26]:
#for each trial, generate ah and ld
rel_AHs = []
rel_LDs = []
abs_AHs = []
abs_LDs = []

for n in np.arange(len(kfiles)):
    
    posture = postures[n]

    svl = key['svl'][n]/100.0 #change SVL to meters
    orig_z = key['orig_z'][n] 
    orig_x = key['orig_x'][n]

    #get the marker spacings based on the number of available markers from the reference sheet
    spacing_columns = all_spacing_columns[:len(posture)-1]        
    spacings = list(key[spacing_columns].loc[n])        
    spacings1 = [x/100 for x in spacings if np.isnan(x) == False] #change cm to m and drop nans

    #calculate AH and LD at both LP and HP
    AH, LD, rel_AH, rel_LD = snake_shape1(posture, spacings1, svl, orig_x, orig_z)
        
    rel_AHs.append(rel_AH)
    rel_LDs.append(rel_LD)
    abs_AHs.append(AH)
    abs_LDs.append(LD)

## Combine into single dataframe

In [27]:
# create list of IDs

IDs_list = list(cp_df['ID'].values)

for i in np.arange(len(position)):
    trial = int(position[i][-25:-22])
    ind = df_list.index(trial)
    IDs_list.append(dp_df['ID'][ind])
    
#create list of svls
SVL_list = list(cp_df['svl'].values)
for i in np.arange(len(position)):
    trial = int(position[i][-25:-22])
    ind = df_list.index(trial)
    SVL_list.append(dp_df['svl'][ind])

#list of trial numbers:
dp_trials = [int(thing[-25:-22]) for thing in position]
trials_list = list(cp_df['tn'].values) + dp_trials

#create list of genera
cp_genus = ["Chrysopelea" for i in np.arange(len(cp_df))]
dp_genus = ['Dendrelaphis' for i in np.arange(len(dp_trials))]
genus_list = cp_genus+dp_genus


In [28]:
cols = ['tn','ID','genus','svl','gscm',
       'hv','zv','hvtf','zvtf',
       'mv','av','lv','mvtf','avtf',
       'over','dtaf','zpaf','zpmx','dft',
       'AH','LD']

all_lists = zip(trials_list, IDs_list,genus_list, SVL_list,cp_gscm+dp_gscm,
             cp_hvar+dp_hvar,cp_zvar+dp_zvar,cp_hvar_tf+dp_hvar_tf,cp_zvar_tf+dp_zvar_tf,
             cp_maxv+dp_maxv,cp_forward+dp_forward,cp_landv+dp_landv,
             cp_maxv_tf+dp_maxv_tf,cp_forward_tf+dp_forward_tf,
             cp_over+dp_over,cp_dtaf+dp_dtaf,cp_zpaf+dp_zpaf,cp_zpmx+dp_zpmx,cp_dft+dp_dft,
               cp_AHs+abs_AHs,cp_LDs+abs_LDs)

kins = pd.DataFrame(all_lists,columns = cols)
kins.head()


,tn,ID,genus,svl,gscm,hv,zv,hvtf,zvtf,mv,...,lv,mvtf,avtf,over,dtaf,zpaf,zpmx,dft,AH,LD
0,44,94,Chrysopelea,85.0,38.705380,0.012648,0.018539,0.003947,0.007770,0.284009,...,0.261874,NaN,NaN,0.002284,0.389585,0.019306,0.019306,0.062976,0.027493,0.008187
1,47,94,Chrysopelea,85.0,43.776015,0.037012,0.072162,0.027074,0.063718,1.152426,...,1.143654,NaN,NaN,0.026699,0.470447,0.035505,0.081239,0.210584,0.184287,0.148782
2,48,94,Chrysopelea,85.0,44.596643,0.040860,0.097483,0.026255,0.096748,1.631175,...,1.628342,NaN,NaN,0.112643,0.559784,-0.000497,0.089859,0.252260,0.150222,0.150719
3,49,94,Chrysopelea,85.0,44.364407,0.011791,0.024126,0.008009,0.006123,0.315210,...,0.314525,NaN,NaN,-0.015649,0.428494,0.021827,0.023582,0.120159,0.060380,0.038553
4,61,89,Chrysopelea,71.5,33.923010,0.013086,0.017914,0.006731,0.012205,0.518157,...,0.451507,NaN,NaN,0.020465,0.356939,0.003759,0.015964,0.236795,0.009567,0.005808


In [29]:
rel_cols = ['gscm',
       'hv','zv','hvtf','zvtf',
       'mv','av','lv','mvtf','avtf',
       'over','dtaf','zpaf','zpmx','dft','AH','LD']

#create relative values
for each in rel_cols:
    new_col_name = each + '_rel'
    kins[new_col_name] = kins[each]/(kins['svl']/100) #change svl to meters; other measures are in meters

In [30]:
#export data to R for statistics
kins.to_csv(cwd+'/all_data_for_stats_withTotalD.csv', index = False)

In [84]:
A['dtaf_rel']*100 - A['gscm_rel']

137    17.485680
138     4.038359
139     1.877468
140     9.203656
141     3.331078
         ...    
250     1.874677
251     6.017051
252     3.790462
253     7.606970
254     4.272755
Length: 118, dtype: float64

# TABLE 1

In [92]:
#overshoot
differences = A['dtaf_rel']*100 - A['gscm_rel']
trials = A['ID']
pairs = zip(differences.values,trials.values)
print(np.mean(differences))
print(np.std(differences))
print(max(differences))

n=0
for each in pairs:
    i,j = each
    if i > 15:
        print(each)
        n = n+1
n

5.526376328651711
5.798360654607126
31.235540806131603
(17.485679559797255, '2018DP01')
(26.900833885143314, '2018DP03')
(31.235540806131603, '2018DC04')
(18.55111743660089, '2018DC04')
(18.26018552962939, '2018DC04')
(19.465198013508825, '2018DC05')
(26.523384430474977, '2018DC05')
(22.747093168148396, '2019DP17')


8

In [32]:
## gaps crossed by individual
kins = kins.astype({'ID':'string'})

for ID in np.unique(kins['ID'].values): 
    print(ID,kins[kins.ID==ID].shape[0])

2018DC04 13
2018DC05 11
2018DP01 4
2018DP02 4
2018DP03 15
2018DP06 11
2019DC07 9
2019DP08 9
2019DP09 3
2019DP10 3
2019DP12 5
2019DP13 2
2019DP15 1
2019DP16 6
2019DP17 7
2019DP18 4
2019DP19 5
2019DP20 6
85 32
88 17
89 31
90 12
94 21
95 24


In [33]:
#gap size ranges by individual
for ID in np.unique(kins['ID'].values): 
    print(ID,min(kins[kins.ID==ID]['gscm_rel']),max(kins[kins.ID==ID]['gscm_rel']))

2018DC04 40.726656854070875 53.5864592875248
2018DC05 48.97290996057948 69.10463562303104
2018DP01 43.352685115553854 48.08739850926988
2018DP02 41.13371037343792 42.983158671978096
2018DP03 41.47266908660566 73.75550618463868
2018DP06 35.41150136621691 51.93519645214992
2019DC07 54.767499831141905 67.23783204715595
2019DP08 33.328895027933584 52.06756815243338
2019DP09 42.14644079289824 45.79301898109456
2019DP10 52.233652506903965 56.294245154274776
2019DP12 44.33463438094458 58.641369720820144
2019DP13 44.98541959969244 49.80960558822571
2019DP15 38.346916950542465 38.346916950542465
2019DP16 37.91598582336037 60.17566095058374
2019DP17 48.019649447553505 65.5465215354434
2019DP18 60.576414786834135 66.2796395056475
2019DP19 58.748316584099 74.03416545633885
2019DP20 40.42252176628308 51.1811531862432
85 45.10816041582324 118.43783623072608
88 38.96185069020119 72.27558534405095
89 47.44476984915433 109.10879599186485
90 37.3483313877368 58.65499369500009
94 45.53574082476151 89.350

In [35]:
#dt ranges by individual
for ID in np.unique(kins['ID'].values): 
    print(ID,min(kins[kins.ID==ID]['svl']),max(kins[kins.ID==ID]['svl']))

2018DC04 77.373 77.373
2018DC05 84.25833333 84.25833333
2018DP01 113.9 113.9
2018DP02 127.4433333 127.4433333
2018DP03 88.97333333 88.97333333
2018DP06 124.3746667 124.3746667
2019DC07 46.19933333 46.19933333
2019DP08 119.094 119.094
2019DP09 109.0 109.0
2019DP10 65.0 65.0
2019DP12 43.0 43.0
2019DP13 55.0 55.0
2019DP15 86.992 86.992
2019DP16 76.98133333 76.98133333
2019DP17 81.23117629 81.23117629
2019DP18 29.534 29.534
2019DP19 30.64466667 30.64466667
2019DP20 89.49533333 89.49533333
85 72.5 72.5
88 95.9 95.9
89 71.5 71.5
90 72.5 72.5
94 85.0 85.0
95 66.7 66.7


In [ ]:
#svls
for ID in np.unique(kins['ID'].values): 
    print(ID,min(kins[kins.ID==ID]['dtaf_rel']),max(kins[kins.ID==ID]['dtaf_rel']))

# REVISION SUBMISSION JEB: ALL OF THE GRAPHS

In [ ]:
from matplotlib.offsetbox import AnchoredText

kins = pd.read_csv(cwd+'/all_data_for_stats.csv')
A = kins[kins['genus']=="Dendrelaphis"]
B = kins[kins['genus'] == "Chrysopelea"]
min_,max_ = kins['svl'].min(),kins['svl'].max()

kins.head()

In [ ]:
IVs = ['gscm_rel']
DVs = ['over_rel','av_rel','mv_rel','lv_rel','hv_rel',
       'zv_rel','zpaf_rel','zpmx_rel','dft_rel','AH_rel','LD_rel']
I_labels = ['Gap Size (%SVL)']
D_labels = ['Overshoot (SVL)', 
            'Average Velocity (SVL/s)', 'Max Velocity (SVL/s)', 'Landing Velocity (SVL/s)',
            'Y Variation, Overall (SVL)', 'Z Variation, Overall (SVL)',
            'Vertical Position at AF (SVL)','Vertical Position Max','Distance from target at AF (SVL)',
           'Arc Height at AF (SVL)','Loop Depth at AF (SVL)']

In [ ]:
#speed parameters; fixed effect only for max and landing; all for average.

# ALL OF THE (original) GRAPHS

In [13]:
from matplotlib.offsetbox import AnchoredText

kins = pd.read_csv(cwd+'/all_data_for_stats.csv')
A = kins[kins['genus']=="Dendrelaphis"]
B = kins[kins['genus'] == "Chrysopelea"]
min_,max_ = kins['svl'].min(),kins['svl'].max()

kins.head()

,tn,ID,genus,svl,gscm,hv,zv,hvtf,zvtf,mv,...,lv_rel,mvtf_rel,avtf_rel,over_rel,dtaf_rel,zpaf_rel,zpmx_rel,dft_rel,AH_rel,LD_rel
0,44,94,Chrysopelea,85.0,38.705380,0.012648,0.018539,0.003947,0.007770,0.284009,...,0.308087,NaN,NaN,0.002687,0.069295,0.022713,0.022713,0.074089,0.032345,0.009632
1,47,94,Chrysopelea,85.0,43.776015,0.037012,0.072162,0.027074,0.063718,1.152426,...,1.345476,NaN,NaN,0.031411,0.275713,0.041771,0.095575,0.247745,0.216809,0.175038
2,48,94,Chrysopelea,85.0,44.596643,0.040860,0.097483,0.026255,0.096748,1.631175,...,1.915696,NaN,NaN,0.132522,0.430223,-0.000585,0.105716,0.296777,0.176732,0.177317
3,49,94,Chrysopelea,85.0,44.364407,0.011791,0.024126,0.008009,0.006123,0.315210,...,0.370030,NaN,NaN,-0.018410,0.114932,0.025679,0.027744,0.141364,0.071036,0.045357
4,61,89,Chrysopelea,71.5,33.923010,0.013086,0.017914,0.006731,0.012205,0.518157,...,0.631479,NaN,NaN,0.028623,0.335943,0.005257,0.022328,0.331182,0.013381,0.008123


In [ ]:
#for annotating particular graphs in case of issues 
ax = plt.gca()
n = B['tn'].values

for t, txt in enumerate(n):
    ax.annotate(txt, (B[IVs[i]][t], B[DVs[d]][t]))

In [14]:
IVs = ['gscm_rel']
DVs = ['over_rel','av_rel','mv_rel','lv_rel','hv_rel',
       'zv_rel','zpaf_rel','zpmx_rel','dft_rel','AH_rel','LD_rel']
I_labels = ['Gap Size (%SVL)']
D_labels = ['Overshoot (SVL)', 
            'Average Velocity (SVL/s)', 'Max Velocity (SVL/s)', 'Landing Velocity (SVL/s)',
            'Y Variation, Overall (SVL)', 'Z Variation, Overall (SVL)',
            'Vertical Position at AF (SVL)','Vertical Position Max','Distance from target at AF (SVL)',
           'Arc Height at AF (SVL)','Loop Depth at AF (SVL)']

In [15]:
#FROM ORIGINAL SUBMISSION average velocity: subplot for each snake with highlight vs grey
plt.rcParams["font.family"] = "Arial"
mp.rcParams['pdf.fonttype'] = 42
mp.rcParams['ps.fonttype'] = 42

min_,max_ = kins['svl'].min(),kins['svl'].max()
norm = plt.Normalize(min_,max_)
cmap = cmr.cosmic_r

pairs = []

for i,j in zip(list(A['svl'].values),list(A['ID'].values)):
    if not (i,j) in pairs:
        pairs.append((i,j))
        
pairs.sort()

a = 3
b = 6

d=1

i=0 #graphing against gap size
fig, ax = plt.subplots(a,b,sharey=True,sharex=True,figsize=(12,8),dpi=100)

for j in range(0,a):
    for k in range(0,b):
        n = b*j+k
        ID = pairs[n][1]

        if ID[-3] == 'P':
            shape = 'o'
            size = 60
        else:
            shape = 'D'
            size = 45


        svl = np.round(pairs[n][0],2)

        C = A[A['ID']==ID]
        ax[j,k].scatter(C[IVs[i]],C[DVs[d]],c=C['svl'],cmap=cmap,
                        marker=shape,norm=norm,edgecolors='k',
                       s=size,alpha=0.75,zorder=1)
        ax[j,k].scatter(A[IVs[i]],A[DVs[d]],c='#D3D3D3',marker='o',edgecolors='#808080',s=size,zorder=0,alpha=0.35)
        ax[j,0].set_ylabel(D_labels[d])
        ax[j,k].set_ylim(0,0.6)
        ax[j,k].set_xlim(25,80)
        ax[j,k].set_xticks([30,40,50,60,70])
        ax[j,k].set_yticks([0.1,0.2,0.3,0.4,0.5])

        # at = AnchoredText(str(svl)+" cm", prop=dict(fontweight='semibold',size=8),
        #                   frameon=True, loc='upper left')
        # at.patch.set_boxstyle("round,pad=0.1,rounding_size=0.2")
        # at.patch.set_edgecolor("k")
        # at.patch.set_facecolor("#D3D3D3")
        # at.patch.set_alpha(0.5)

        # ax[j,k].add_artist(at)


plt.subplots_adjust(wspace=0, hspace=0, bottom = 0.2)
plt.figtext(0.475,0.14,I_labels[i])


Text(0.475, 0.14, 'Gap Size (%SVL)')

In [130]:
#average velocity: size pooled with prediction lines
plt.rcParams["font.family"] = "Arial"
mp.rcParams['pdf.fonttype'] = 42
mp.rcParams['ps.fonttype'] = 42

min_,max_ = kins['svl'].min(),kins['svl'].max()
norm = plt.Normalize(min_,max_)
cmap = cmr.cosmic_r

pairs = []

for i,j in zip(list(A['svl'].values),list(A['ID'].values)):
    if not (i,j) in pairs:
        pairs.append((i,j))
        
pairs.sort()

#import effects
effects = pd.read_csv(cwd+'/av_effects.csv')
fig, ax = plt.subplots(1,3,sharey=True,sharex=True,figsize=(12,8),dpi=100)
i=0
d=1

for each in pairs:
    ID = each[1]
    svl = each[0]
    C = A[A['ID']==ID]
    
    #SET MARKER SHAPE: species
    if ID[-3] == 'P':
        shape = 'o'
    else:
        shape = 'D'
        
    #SET SIZE CATEGORY: svl
    if svl <= 65:
        j = 0
    elif svl < 100:
        j = 1
    else:
        j = 2
        
    #get gap size range; predicted effect
    if ID != "2019DP13" and ID!="2019DP15":
        gs_min = min(C['gscm_rel'])
        gs_max = max(C['gscm_rel'])
        gs_range = np.arange(gs_min,gs_max,1)
        params = effects[effects['ID']==ID].values[0]
        preds = [params[2]*(k/100-0.333289)+params[3]*svl+params[1] for k in gs_range]
        ax[j].plot(gs_range,preds,c=cmap(norm(svl)),lw=3)

    #PLOT DATA
    ax[j].plot(C[IVs[i]],C[DVs[d]],shape,color=cmap(norm(svl)),ms=10,alpha=0.5,label=str(np.round(svl,1))+'cm')
    ax[0].set_ylabel(D_labels[d])
    ax[j].legend(ncol=2)

plt.subplots_adjust(wspace=0, hspace=0, bottom = 0.2)
plt.figtext(0.475,0.14,I_labels[i])

sizes=["Small snakes","Medium snakes","Large snakes"]
for i in [0,1,2]:
    ax[i].set_title(sizes[i])


## GRAPHS

In [167]:
# max,  and landing velocity: all data with lines?

plt.rcParams["font.family"] = "Arial"
mp.rcParams['pdf.fonttype'] = 42
mp.rcParams['ps.fonttype'] = 42

min_,max_ = kins['svl'].min(),kins['svl'].max()
norm = plt.Normalize(min_,max_)
cmap = cmr.cosmic_r

pairs = []

for i,j in zip(list(A['svl'].values),list(A['ID'].values)):
    if not (i,j) in pairs:
        pairs.append((i,j))
        
pairs.sort()

#import effects
effects = [pd.read_csv(cwd+'/mv_effects.csv'),pd.read_csv(cwd+'/lv_effects.csv')]
fig, ax = plt.subplots(1,3,figsize=(12,8),dpi=100)
i=0
dvals = [2,3]

for each in pairs:
    for m in [0,1]:  
        d = dvals[m]
        ID = each[1]
        svl = each[0]
        C = A[A['ID']==ID]

        #SET MARKER SHAPE: species
        if ID[-3] == 'P':
            shape = 'o'
        else:
            shape = 'D'

        eff = effects[m]
        #get gap size range; predicted effect
        if ID != "2019DP13" and ID!="2019DP15":
            gs_min = min(C['gscm_rel'])
            gs_max = max(C['gscm_rel'])
            gs_range = np.arange(gs_min,gs_max,1)
            params = eff[eff['ID']==ID].values[0]
            preds = [params[2]*(k/100-0.333289)+params[3]*svl+params[1] for k in gs_range]
            ax[m].plot(gs_range,preds,'--',c=cmap(norm(svl)),lw=3)

        ax[m].plot(C[IVs[i]],C[DVs[d]],shape,color=cmap(norm(svl)),ms=10,label=str(np.round(svl,1))+'cm')
        ax[m].set_ylabel(D_labels[d])
        ax[m].set_xlabel(I_labels[i])
        ax[m].set_ylim(0,3.8)
    
cb1 = mp.colorbar.ColorbarBase(ax[2],cmap=cmap,
                                norm=norm,
                                orientation='vertical',
                               label = 'Snout-venth length (cm)')

legend_elements = [Line2D([0], [0], marker='o', color='#FFFFFF', label="D. punctulatus",
                          markerfacecolor='#808080', markersize=10),
                   Line2D([0], [0], marker='D', color='#FFFFFF', label='D. calligastra',
                          markerfacecolor='#808080', markersize=10)]
plt.legend(handles=legend_elements, loc='upper right')

In [103]:
#total distance traveled: CONNECTED TO GAP SIZE (see ZPOS special graph).
IVs = ['gscm_rel', 'dtaf_rel']
DVs = ['dtaf_rel','over_rel','av_rel','mv_rel','lv_rel','hv_rel',
       'zv_rel','zpaf_rel','zpmx_rel','dft_rel','AH_rel','LD_rel']
I_labels = ['Gap Size (%SVL)','Distance traveled from AF (SVLs)']
D_labels = ['Total Distance Traveled (SVL)','Overshoot (SVL)', 
            'Average Velocity (SVL/s)', 'Max Velocity (SVL/s)', 'Landing Velocity (SVL/s)',
            'Y Variation, Overall (SVL)', 'Z Variation, Overall (SVL)',
            'Vertical Position at AF (SVL)','Vertical Position Max','Distance from target at AF (SVL)',
           'Arc Height at AF (SVL)','Loop Depth at AF (SVL)']

plt.rcParams["font.family"] = "Arial"
mp.rcParams['pdf.fonttype'] = 42
mp.rcParams['ps.fonttype'] = 42

min_,max_ = kins['svl'].min(),kins['svl'].max()
norm = plt.Normalize(min_,max_)
cmap = cmr.cosmic_r

pairs = []

for i,j in zip(list(A['svl'].values),list(A['ID'].values)):
    if not (i,j) in pairs:
        pairs.append((i,j))
        
pairs.sort()

fig, ax = plt.subplots(2,2,sharex = "col",sharey="col",figsize=(12,8),dpi=100)
i=0
d=0

for each in pairs:
    ID = each[1]
    svl = each[0]
    C = A[A['ID']==ID]
    
    #SET MARKER SHAPE: species
    if ID[-3] == 'P':
        shape = 'o'
    else:
        shape = 'D'
        

    #PLOT DATA: all distances traveled vs gap size
    ax[0,0].plot(C[IVs[i]],C[DVs[d]]*100,shape,color=cmap(norm(svl)),ms=10,alpha=0.5,label=str(np.round(svl,1))+'cm')

    #PLOT DATA: largest distances traveled, with associated gap size. 
    x=svl
    y0 = np.max(C[DVs[d]])*100
    index = np.argmax(C[DVs[d]])
    y1 = C[IVs[i]].iloc[index]
    
    ax[0,1].scatter(x,y0,color=cmap(norm(svl)),marker=shape,s=50,alpha=1,zorder=2)
    ax[0,1].scatter(x,y1,color=cmap(norm(svl)),marker=shape,s=50,alpha=0.5,zorder=3)
    ax[0,1].plot((x,x),(y0, y1),linestyle='dotted',c='#808080',zorder=1)
    

ax[0,0].plot(np.arange(40,140),np.arange(40,140),'--',c="#D3D3D3",zorder=0)
ax[1,0].plot(np.arange(40,140),np.arange(40,140),'--',c="#D3D3D3")
#Chrysopelea   
pairs = []

for i,j in zip(list(B['svl'].values),list(B['ID'].values)):
    if not (i,j) in pairs:
        pairs.append((i,j))
        
pairs.sort()

i=0
d=0

for each in pairs:
    ID = each[1]
    svl = each[0]
    C = B[B['ID']==ID]
    shape = 'o'
        

    #PLOT DATA
    ax[1,0].plot(C[IVs[i]],C[DVs[d]]*100,shape,color=cmap(norm(svl)),ms=10,alpha=0.5,label=str(np.round(svl,1))+'cm')
    
    #Plot max data
    x=svl
    y0 = np.max(C[DVs[d]])*100
    index = np.argmax(C[DVs[d]])
    y1 = C[IVs[i]].iloc[index]
    
    ax[1,1].scatter(x,y0,color=cmap(norm(svl)),marker=shape,s=50,alpha=1,zorder=2)
    ax[1,1].scatter(x,y1,color=cmap(norm(svl)),marker=shape,s=50,alpha=0.5,zorder=3)
    ax[1,1].plot((x,x),(y0, y1),linestyle='dotted',c='#808080',zorder=1)

ax[0,0].set_ylabel("Total Distance Traveled (%SVL)")
ax[1,0].set_xlabel("Gap Size (%SVL)")
ax[1,1].set_xlabel("SVL (cm)")
ax[0,1].set_title("Largest distance traveled for each snake")
ax[1,1].set_ylim(30,150)


(30.0, 150.0)

In [166]:
#posture and extent: all data with lines?

plt.rcParams["font.family"] = "Arial"
mp.rcParams['pdf.fonttype'] = 42
mp.rcParams['ps.fonttype'] = 42

min_,max_ = kins['svl'].min(),kins['svl'].max()
norm = plt.Normalize(min_,max_)
cmap = cmr.cosmic_r

pairs = []

for i,j in zip(list(A['svl'].values),list(A['ID'].values)):
    if not (i,j) in pairs:
        pairs.append((i,j))
        
pairs.sort()

#import effects
effects = [pd.read_csv(cwd+'/hv_effects.csv'),pd.read_csv(cwd+'/zv_effects.csv'),
                pd.read_csv(cwd+'/AH_effects.csv'),pd.read_csv(cwd+'/LD_effects.csv')]
fig, ax = plt.subplots(2,2,sharey='all',figsize=(12,8),dpi=100)
i=0
dvals = [4,5,9,10]
ax_vals = [(0,0),(0,1),(1,0),(1,1)]

for each in pairs:
    for m in [0,1,2,3]:  
        d = dvals[m]
        ID = each[1]
        svl = each[0]
        C = A[A['ID']==ID]

        #SET MARKER SHAPE: species
        if ID[-3] == 'P':
            shape = 'o'
        else:
            shape = 'D'

        eff = effects[m]
        #get gap size range; predicted effect
        if ID != "2019DP13" and ID!="2019DP15":
            gs_min = min(C['gscm_rel'])
            gs_max = max(C['gscm_rel'])
            gs_range = np.arange(gs_min,gs_max,1)
            params = eff[eff['ID']==ID].values[0]
            preds = [params[2]*(k/100-0.333289)+params[3]*svl+params[1] for k in gs_range]
            ax[ax_vals[m]].plot(gs_range,preds,'--',c=cmap(norm(svl)),lw=3)

        ax[ax_vals[m]].plot(C[IVs[i]],C[DVs[d]],shape,color=cmap(norm(svl)),alpha=0.5,ms=5,label=str(np.round(svl,1))+'cm')
        ax[ax_vals[m]].set_ylabel(D_labels[d])
        ax[ax_vals[m]].set_xlabel(I_labels[i])


legend_elements = [Line2D([0], [0], marker='o', color='#FFFFFF', label="D. punctulatus",
                          markerfacecolor='#808080', markersize=10),
                   Line2D([0], [0], marker='D', color='#FFFFFF', label='D. calligastra',
                          markerfacecolor='#808080', markersize=10)]
plt.legend(handles=legend_elements, loc='upper right')

## Absolute versions (meters)

In [40]:
#all combinations of data columns: absolute versions
cmap = cmr.cosmic_r  

IVs = ['gscm', 'dtaf']
DVs = ['over','av','mv','lv','mvtf','avtf','hv','zv','hvtf','zvtf','zpaf','dft','AH','LD']
I_labels = ['Gap Size (m)','Distance Traveled from AF (m)']
D_labels = ['Overshoot (m)', 'Average Velocity (m/s)', 'Max Velocity (m/s)', 'Landing Velocity (m/s)',
              'Max Velocity; NC (m/s)','Avg. Velocity; NC (m/s)',
            'Y Variation, Overall (m)', 'Z Variation, Overall (m)', 'Y Variation, NC (m)',
                   'Z Variation, NC (m)','Position relative to branch at AF (m)',
           'Distance from target at AF (m)','Arc Height at AF (m)','Loop Depth at AF (m)']

A = kins[kins['genus']=="Dendrelaphis"]
B = kins[kins['genus'] == "Chrysopelea"]
min_,max_ = kins['svl'].min(),kins['svl'].max()

norm = plt.Normalize(min_,max_)

ylims = []
xlims = []

for i in np.arange(len(IVs)):
    for d in np.arange(len(DVs)):
        plt.figure()
        plt.scatter(A[IVs[i]],A[DVs[d]],c=A['svl'],cmap='winter',marker='D',norm=norm,label='Dendrealphis')
        plt.scatter(B[IVs[i]],B[DVs[d]],c=B['svl'],cmap='winter',marker='o',norm=norm,label = 'Chrysopelea')

        plt.colorbar().set_label('Snout-Vent Length (cm)',rotation=-270)
        plt.legend()
        
        plt.xlabel(I_labels[i])
        plt.ylabel(D_labels[d])
        
        axes = plt.gca()
        y_min, y_max = axes.get_ylim()
        ylims.append([y_min,y_max])      
        x_min, x_max = axes.get_xlim()
        xlims.append([x_min,x_max])

        plt.savefig(cwd+"/Graphs/Absolute Graphs/Comp_"+IVs[i]+'_'+DVs[d]+'.pdf')
        plt.close()
                    
for i in np.arange(len(IVs)):
    for d in np.arange(len(DVs)):
        
        n=len(DVs)*i+d
                    
        plt.figure()
        plt.scatter(A[IVs[i]],A[DVs[d]],c=A['svl'],cmap=cmap,marker='D',norm=norm,label='Dendrelaphis')

        plt.ylim(ylims[n])
        plt.xlim(xlims[n])
        
        plt.colorbar().set_label('Snout-Vent Length (cm)',rotation=-270)
        plt.legend()
        
        plt.xlabel(I_labels[i])
        plt.ylabel(D_labels[d])

        plt.savefig(cwd+"/Graphs/Absolute Graphs/DP_"+IVs[i]+'_'+DVs[d]+'.pdf')
        plt.close()


## Relative to SVL

### Summaries where there is no GS relationship

In [6]:
IVs = ['gscm_rel', 'dtaf_rel']
DVs = ['over_rel','av_rel','mv_rel','lv_rel','hv_rel',
       'zv_rel','zpaf_rel','zpmx_rel','dft_rel','AH_rel','LD_rel']
I_labels = ['Gap Size (%SVL)','Distance traveled from AF (SVLs)']
D_labels = ['Overshoot (SVL)', 
            'Average Velocity (SVL/s)', 'Max Velocity (SVL/s)', 'Landing Velocity (SVL/s)',
            'Y Variation, Overall (SVL)', 'Z Variation, Overall (SVL)',
            'Vertical Position at AF (SVL)','Vertical Position Max','Distance from target at AF (SVL)',
           'Arc Height at AF (% SVL)','Loop Depth at AF (SVL)']

A = kins[kins['genus']=="Dendrelaphis"]
B = kins[kins['genus'] == "Chrysopelea"]
min_,max_ = kins['svl'].min(),kins['svl'].max()
norm = plt.Normalize(min_,max_)

pairs = []

for i,j in zip(list(A['svl'].values),list(A['ID'].values)):
    if not (i,j) in pairs:
        pairs.append((i,j))
        
        
pairs.sort()

In [14]:
plt.rcParams["font.family"] = "Arial"
mp.rcParams['pdf.fonttype'] = 42
mp.rcParams['ps.fonttype'] = 42
cmap = cmr.cosmic_r


for d in np.arange(len(DVs)):

    i=0 #graphing against gap size
    fig, ax = plt.subplots(figsize=(12,8),dpi=100)

    for n in np.arange(len(pairs)):
        ID = pairs[n][1]   
        svl = pairs[n][0]
        C = A[A['ID']==ID]
        
        
        #SET MARKER SYMBOL: species
        if ID[-3] == 'P':
            shape = 'o'
            size = 60
        else:
            shape = 'D'
            size = 45
        
        #AVG VALS
        xval = np.mean(C[IVs[i]])
        yval = np.mean(C[DVs[d]])

        #RANGES
        val_std = np.std(C[DVs[d]])
        xmin = abs(xval-np.min(C[IVs[i]]))
        xmax = abs(xval-np.max(C[IVs[i]]))
        
        gs = np.empty((2,1))
        gs[0,0] = xmin
        gs[1,0] = xmax
        
        ax.scatter(xval, yval, marker=shape,s=size,color=cmap(norm(svl)),
                   edgecolor='k',label=str(np.round(svl,1))+' cm',zorder=2)
        
        ax.errorbar(xval, yval, xerr=gs, yerr=val_std, 
                    fmt='none', color=cmap(norm(svl)),
                    capsize=2,zorder=1)

            

    ax.set_ylabel(D_labels[d])
    ax.set_xlabel(I_labels[i])
    ax.legend(loc="upper left",ncol=3)

    plt.savefig("/Users/grahmich/Desktop/Kinematics Paper/Graphs/IndAvgs_"+IVs[i]+'_'+DVs[d]+'_Dend.pdf')
    plt.close()


### Summary where there is no body size effect: average with s.d. for each gap size?

In [71]:
gs_min = np.min(A['gscm_rel'])
gs_max = np.max(A['gscm_rel'])

bins = np.arange(int(gs_min),int(gs_max),5)
   
for d in np.arange(len(DVs)):
    fig, ax = plt.subplots(figsize=(12,8),dpi=100)
    
    for i in np.ara
    plt.scatter(A[IVs[i]],A[DVs[d]],c=A['svl'],cmap=cmap,norm=norm,zorder=1,alpha=0.03)

    for j in np.arange(len(bins)-1):
        C = A[A['gscm_rel']<bins[j+1]]
        C = C[C['gscm_rel']>bins[j]]
        yval = np.mean(C[DVs[d]])
        xval = np.mean(C['gscm_rel'])

        #RANGES
        val_std = np.std(C[DVs[d]])
        xmin = abs(xval-np.min(C['gscm_rel']))
        xmax = abs(xval-np.max(C['gscm_rel']))

        gs = np.empty((2,1))
        gs[0,0] = xmin
        gs[1,0] = xmax

        ax.scatter(xval, yval,color="k",zorder=3,s=100)
        

        ax.errorbar(xval, yval, xerr=gs, yerr=val_std, 
                    fmt='none', color="k",
                    capsize=2,zorder=2,
                   linewidth=2)

    C = A[A['gscm_rel']>bins[-1]]
    yval = np.mean(C[DVs[d]])
    xval = np.mean(C['gscm_rel'])

    #RANGES
    val_std = np.std(C[DVs[d]])
    xmin = abs(xval-np.min(C['gscm_rel']))
    xmax = abs(xval-np.max(C['gscm_rel']))

    gs = np.empty((2,1))
    gs[0,0] = xmin
    gs[1,0] = xmax

    ax.scatter(xval, yval,color="k",zorder=2)

    ax.errorbar(xval, yval, xerr=gs, yerr=val_std, 
                fmt='none', color="#808080",
                capsize=2,zorder=1)

    ax.set_ylabel(D_labels[d])
    ax.set_xlabel(I_labels[i])

    plt.savefig("/Users/grahmich/Desktop/Kinematics Paper/Graphs/GSAvgs_"+IVs[i]+'_'+DVs[d]+'_Dend_wIndData.pdf')
    plt.close()
        

### By individual but pooled into size categories

In [41]:
IVs = ['gscm_rel', 'dtaf_rel']
DVs = ['dtaf_rel','over_rel','av_rel','mv_rel','lv_rel','hv_rel',
       'zv_rel','zpaf_rel','zpmx_rel','dft_rel','AH_rel','LD_rel']
I_labels = ['Gap Size (%SVL)','Distance traveled from AF (SVLs)']
D_labels = ['Total Distance Traveled (SVL)','Overshoot (SVL)', 
            'Average Velocity (SVL/s)', 'Max Velocity (SVL/s)', 'Landing Velocity (SVL/s)',
            'Y Variation, Overall (SVL)', 'Z Variation, Overall (SVL)',
            'Vertical Position at AF (SVL)','Vertical Position Max','Distance from target at AF (SVL)',
           'Arc Height at AF (SVL)','Loop Depth at AF (SVL)']

In [42]:
pairs = []

for i,j in zip(list(A['svl'].values),list(A['ID'].values)):
    if not (i,j) in pairs:
        pairs.append((i,j))
        
pairs.sort()

In [43]:
plt.rcParams["font.family"] = "Arial"
mp.rcParams['pdf.fonttype'] = 42
mp.rcParams['ps.fonttype'] = 42
min_,max_ = kins['svl'].min(),kins['svl'].max()
norm = plt.Normalize(min_,max_)
cmap = cmr.cosmic_r
shapes = ['o','^','<','>','o','^','<','>','o','^','<','>','o','^','<','>','o','^','<','>']
t=0

for d in [0]:

    i=0 #graphing against gap size
    fig, ax = plt.subplots(1,3,sharey=True,sharex=True,figsize=(12,8),dpi=100)

    for n in np.arange(len(pairs)):
        ID = pairs[n][1]  
        svl = np.round(pairs[n][0],2)
        shape=shapes[n]
        
        #SET MARKER FILL: species
        if ID[-3] == 'P':
            facecolor = cmap(norm(svl))
            edgecolor = cmap(norm(svl))
        else:
            facecolor = 'none'
            edgecolor = cmap(norm(svl))
        
        #SET SIZE CATEGORY: svl
        if svl <= 65:
            j = 0
        elif svl < 100:
            j = 1
        else:
            j = 2
            
            
        #PLOT DATA
        C = A[A['ID']==ID]
        ax[j].plot(C[IVs[i]],C[DVs[d]],shape,mfc=facecolor,mec=edgecolor,ms=10,label=str(svl)+'cm')
        ax[0].set_ylabel(D_labels[d])
        ax[j].legend(loc="upper left")
        t=t+1

    plt.subplots_adjust(wspace=0, hspace=0, bottom = 0.2)
    plt.figtext(0.475,0.14,I_labels[i])

    plt.savefig("/Users/grahmich/Desktop/Kinematics Paper/Graphs/SizePooled_"+IVs[i]+'_'+DVs[d]+'_Dend.pdf')
    plt.close()


### All Data Together

In [14]:
#all combinations of data columns: relative versions
cmap = cmr.cosmic_r  
plt.rcParams["font.family"] = "Arial"
mp.rcParams['pdf.fonttype'] = 42
mp.rcParams['ps.fonttype'] = 42

IVs = ['gscm_rel']
DVs = ['dtaf_rel','over_rel','av_rel','mv_rel','lv_rel','hv_rel',
       'zv_rel','zpaf_rel','zpmx_rel','dft_rel','AH_rel','LD_rel']
I_labels = ['Gap Size (%SVL)']
D_labels = ['Total Distance Traveled (SVL)','Overshoot (SVL)', 
            'Average Velocity (SVL/s)', 'Max Velocity (SVL/s)', 'Landing Velocity (SVL/s)',
            'Y Variation, Overall (SVL)', 'Z Variation, Overall (SVL)',
            'Vertical Position at AF (SVL)','Vertical Position Max','Distance from target at AF (SVL)',
           'Arc Height at AF (SVL)','Loop Depth at AF (SVL)']

A = kins[kins['genus']=="Dendrelaphis"]
B = kins[kins['genus'] == "Chrysopelea"]
min_,max_ = kins['svl'].min(),kins['svl'].max()

norm = plt.Normalize(min_,max_)

ylims = []
xlims = []

for i in np.arange(len(IVs)):
    for d in np.arange(len(DVs)):
        plt.figure()
        plt.scatter(A[IVs[i]],A[DVs[d]],c=A['svl'],cmap=cmap,marker='o',norm=norm)
        
        for k in np.arange(len(B[IVs[i]])):
            svl = B['svl'][k]
            col = cmap(norm(svl))
            plt.plot(B[IVs[i]][k],B[DVs[d]][k],marker='o',mfc='none',mec=col)

        
        plt.colorbar().set_label('Snout-Vent Length (cm)',rotation=-270)
        plt.xlabel(I_labels[i])
        plt.ylabel(D_labels[d])
        
        
        legend_elements = [Line2D([0], [0], marker='o', color='#FFFFFF', label="Dendrelaphis",
                          markerfacecolor='#808080', markersize=10),
                   Line2D([0], [0], marker='o', color='#FFFFFF', label='Chrysopelea',
                          markerfacecolor='none', markeredgecolor = '#808080', markersize=10)]
        plt.legend(handles=legend_elements, loc='upper right')
        
        
        axes = plt.gca()
        y_min, y_max = axes.get_ylim()
        ylims.append([y_min,y_max])      
        x_min, x_max = axes.get_xlim()
        xlims.append([x_min,x_max])

        plt.savefig(cwd+"/Graphs/Comp_"+IVs[i]+'_'+DVs[d]+'.pdf')
        plt.close()
                    
for i in np.arange(len(IVs)):
    for d in np.arange(len(DVs)):
        
        n=len(DVs)*i+d
                    
        plt.figure()
        plt.scatter(A[IVs[i]],A[DVs[d]],c=A['svl'],cmap=cmap,marker='D',norm=norm,label='Dendrelaphis')

        plt.ylim(ylims[n])
        plt.xlim(xlims[n])
        
        plt.colorbar().set_label('Snout-Vent Length (cm)',rotation=-270)
        plt.legend()
        
        plt.xlabel(I_labels[i])
        plt.ylabel(D_labels[d])

        plt.savefig(cwd+"/Graphs/DP_"+IVs[i]+'_'+DVs[d]+'.pdf')
        plt.close()


### Graphs by individual -- Dendrelaphis

In [72]:
cmap = cmr.cosmic_r  

IVs = ['gscm_rel', 'dtaf_rel']
DVs = ['over_rel','av_rel','mv_rel','lv_rel','hv_rel',
       'zv_rel','zpaf_rel','zpmx_rel','dft_rel','AH_rel','LD_rel']
I_labels = ['Gap Size (%SVL)','Distance traveled from AF (SVLs)']
D_labels = ['Overshoot (SVL)', 
            'Average Velocity (SVL/s)', 'Max Velocity (SVL/s)', 'Landing Velocity (SVL/s)',
            'Y Variation, Overall (SVL)', 'Z Variation, Overall (SVL)',
            'Vertical Position at AF (SVL)','Vertical Position Max','Distance from target at AF (SVL)',
           'Arc Height at AF (SVL)','Loop Depth at AF (SVL)']

A = kins[kins['genus']=="Dendrelaphis"]
B = kins[kins['genus'] == "Chrysopelea"]
min_,max_ = kins['svl'].min(),kins['svl'].max()

norm = plt.Normalize(min_,max_)

In [ ]:
pairs = []

for i,j in zip(list(A['svl'].values),list(A['ID'].values)):
    if not (i,j) in pairs:
        if not j == "2019DP15" and not j == "2019DP13":
            pairs.append((i,j))
        
        
pairs.sort()

a = 4
b = 4

for d in np.arange(len(DVs)):

    i=0 #graphing against gap size
    fig, ax = plt.subplots(a,b,sharey=True,sharex=True,figsize=(12,8),dpi=100)

    for j in range(0,a):
        for k in range(0,b):
            n = b*j+k
            ID = pairs[n][1]
            
            if ID[-3] == 'P':
                shape = 'o'
                size = 60
            else:
                shape = 'D'
                size = 45
            
            
            svl = np.round(pairs[n][0],2)

            C = A[A['ID']==ID]
            ax[j,k].scatter(C[IVs[i]],C[DVs[d]],c=C['svl'],cmap=cmap,
                            marker=shape,norm=norm,edgecolors='#808080',
                           s=size)
            ax[j,0].set_ylabel(D_labels[d])

            at = AnchoredText(str(svl)+" cm", prop=dict(fontweight='semibold',size=8),
                              frameon=True, loc='upper left')
            at.patch.set_boxstyle("round,pad=0.1,rounding_size=0.2")
            at.patch.set_edgecolor("k")
            at.patch.set_facecolor("#D3D3D3")
            at.patch.set_alpha(0.5)

            ax[j,k].add_artist(at)


    plt.subplots_adjust(wspace=0, hspace=0, bottom = 0.2)
    plt.figtext(0.475,0.14,I_labels[i])

IndexError: list index out of range

In [76]:
plt.rcParams["font.family"] = "Arial"
mp.rcParams['pdf.fonttype'] = 42
mp.rcParams['ps.fonttype'] = 42

a = 4
b = 4

for d in np.arange(len(DVs)):

    i=0 #graphing against gap size
    fig, ax = plt.subplots(a,b,sharey=True,sharex=True,figsize=(12,8),dpi=100)

    for j in range(0,a):
        for k in range(0,b):
            n = b*j+k
            ID = pairs[n][1]
            
            if ID[-3] == 'P':
                shape = 'o'
                size = 60
            else:
                shape = 'D'
                size = 45
            
            
            svl = np.round(pairs[n][0],2)

            C = A[A['ID']==ID]
            ax[j,k].scatter(C[IVs[i]],C[DVs[d]],c=C['svl'],cmap=cmap,
                            marker=shape,norm=norm,edgecolors='#808080',
                           s=size)
            ax[j,0].set_ylabel(D_labels[d])

            at = AnchoredText(str(svl)+" cm", prop=dict(fontweight='semibold',size=8),
                              frameon=True, loc='upper left')
            at.patch.set_boxstyle("round,pad=0.1,rounding_size=0.2")
            at.patch.set_edgecolor("k")
            at.patch.set_facecolor("#D3D3D3")
            at.patch.set_alpha(0.5)

            ax[j,k].add_artist(at)


    plt.subplots_adjust(wspace=0, hspace=0, bottom = 0.2)
    plt.figtext(0.475,0.14,I_labels[i])

    plt.savefig("/Users/grahmich/Desktop/Ind_"+IVs[i]+'_'+DVs[d]+'_Dend.pdf')
    plt.close()


### Special graph: ZPaf vs ZPmax

In [194]:
#special graph for Z position 
plt.rcParams["font.family"] = "Arial"
mp.rcParams['pdf.fonttype'] = 42
mp.rcParams['ps.fonttype'] = 42

a = 3
b = 6

i = 0
fig, ax = plt.subplots(a,b,sharey=True,sharex=True,figsize=(12,8),dpi=100)
for j in range(0,a):
    for k in range(0,b):
        n = b*j+k
        ID = pairs[n][1]

        if ID[-3] == 'P':
            shape = 'o'
            size = 60
        else:
            shape = 'D'
            size = 45

        svl = np.round(pairs[n][0],2)

        C = A[A['ID']==ID]
        x = C[IVs[i]]
        y0 = C[DVs[6]]
        y1 = C[DVs[7]]

        ax[j,k].axhline(0,c='#808080',zorder=0)
        ax[j,k].scatter(x,y0,c='k',marker=shape,s=size,alpha=1,zorder=2,edgecolors='w',linewidth=0.2)
        ax[j,k].scatter(x,y1,c='k',marker=shape,s=size,alpha=0.5,zorder=3)
        ax[j,k].plot((x,x),(y0, y1),'--',c='#D3D3D3',zorder=1)
        
        at = AnchoredText(str(svl)+" cm", prop=dict(fontweight='semibold',size=12),
                          frameon=True, loc='upper left')
        at.patch.set_boxstyle("round,pad=0.1,rounding_size=0.2")
        at.patch.set_edgecolor("k")
        at.patch.set_facecolor("#D3D3D3")
        at.patch.set_alpha(0.5)

        ax[j,k].add_artist(at)
    

ax[1,0].set_ylabel("Vertical Position (SVL)",fontsize=10)

plt.subplots_adjust(wspace=0, hspace=0, bottom = 0.2)
plt.figtext(0.475,0.14,I_labels[i])

legend_elements = [Line2D([0], [0], marker='o', color='k', label="AF",
                      markerfacecolor='k', markersize=10), 
                   Line2D([0], [0], marker='o', color='k', label='Max',
                      markerfacecolor='#808080', markeredgecolor = 'k',alpha=0.5, markersize=10)]
fig.legend(handles=legend_elements, loc='upper center',ncol=2)

### Graphs by individual -- Chrysopelea

In [38]:
cpairs = []
plt.rcParams["font.family"] = "Arial"

for size,ind in zip(list(B['svl'].values),list(B['ID'].values)):
    if not (size,ind) in cpairs:
        cpairs.append((size,ind))
        
        
cpairs.sort()

for i in np.arange(len(IVs)):
    for d in np.arange(len(DVs)):
        fig, ax = plt.subplots(2,3,sharey=True,sharex=True,figsize=(16,9),dpi=100)

        for j in range(0,2):
            for k in range(0,3):
                n = 3*j+k
                ID = cpairs[n][1]
                svl = np.round(cpairs[n][0],2)

                C = B[B['ID']==ID]
                ax[j,k].scatter(C[IVs[i]],C[DVs[d]],c=C['svl'],cmap=cmap,marker='D',norm=norm)
                ax[j,k].set_title(str(svl) +" cm")
                ax[j,0].set_ylabel(D_labels[d])

        ax[1,1].set_xlabel(I_labels[i])
        fig.suptitle("Chrysopelea")

        plt.savefig(cwd+"/Graphs/Individual Breakouts/"+IVs[i]+'_'+DVs[d]+'_CP.png')
        plt.close()

### Special graph, Chrysopelea: ZPaf vs ZPmax

In [188]:
cpairs = []

for size,ind in zip(list(B['svl'].values),list(B['ID'].values)):
    if not (size,ind) in cpairs:
        cpairs.append((size,ind))
        
        
cpairs.sort()

In [193]:
#special graph for Z position 
plt.rcParams["font.family"] = "Arial"
a = 2
b = 3

i = 0
fig, ax = plt.subplots(a,b,sharey=True,sharex=True,figsize=(12,8),dpi=100)

for j in range(0,a):
    for k in range(0,b):
        n = b*j+k
        ID = cpairs[n][1]

        svl = np.round(cpairs[n][0],2)

        C = B[B['ID']==ID]
        x = C[IVs[i]]
        y0 = C[DVs[6]]
        y1 = C[DVs[7]]
        ax[j,k].axhline(0,c='#808080',zorder=0)
        ax[j,k].scatter(x,y0,c='k',marker='o',s=60,alpha=1,zorder=2,edgecolors='w',linewidth=0.2)
        ax[j,k].scatter(x,y1,c='k',marker='o',s=60,alpha=0.5,zorder=3)
        
        ax[j,k].plot((x,x),(y0, y1),'--',c='#D3D3D3',zorder=1)
        
        
        

        at = AnchoredText(str(svl)+" cm", prop=dict(fontweight='semibold',size=12),
                          frameon=True, loc='upper left')
        at.patch.set_boxstyle("round,pad=0.1,rounding_size=0.2")
        at.patch.set_edgecolor("k")
        at.patch.set_facecolor("#D3D3D3")
        at.patch.set_alpha(0.5)

        ax[j,k].add_artist(at)

ax[1,0].set_ylabel("Vertical Position (SVL)")
plt.subplots_adjust(wspace=0, hspace=0, bottom = 0.2)
plt.figtext(0.475,0.14,I_labels[i],)

legend_elements = [Line2D([0], [0], marker='o', color='k', label="AF",
                      markerfacecolor='k', markersize=10),
               Line2D([0], [0], marker='o', color='k', label='Max',
                      markerfacecolor='#808080', markeredgecolor = 'k',alpha=0.5, markersize=10)]
fig.legend(handles=legend_elements, loc='upper center',ncol=2)
plt.figtext(0.475,0.9,"Chrysopelea")

Text(0.475, 0.9, 'Chrysopelea')

### Making every possible contrast

In [4]:
cmap = cmr.ember_r  
cmap1 = cmr.sapphire_r

variables = ['gscm_rel', 'dtaf_rel','av_rel','mv_rel','lv_rel',
             'hv_rel','zv_rel','zpaf_rel','zpmx_rel','dft_rel','AH_rel','LD_rel']

labels = ['Gap Size (%SVL)','Distance traveled from AF (SVLs)','Average Velocity (SVL/s)', 
          'Max Velocity (SVL/s)', 'Landing Velocity (SVL/s)',
            'Y Variation, Overall (SVL)', 'Z Variation, Overall (SVL)',
          'Vertical Position at AF (SVL)','Vertical Position Max (SVL)','Distance from target at AF (SVL)',
           'Arc Height at AF (SVL)','Loop Depth at AF (SVL)']

ablabels = ['gscm', 'dtaf','av','mv','lv','hv','zv','zpaf','zpmx','dft','AH','LD']


A = kins[kins['genus']=="Dendrelaphis"]
B = kins[kins['genus'] == "Chrysopelea"]

dmx = max(A['gscm_rel'])
B = kins[kins['gscm_rel'] <= dmx]

min_,max_ = kins['svl'].min(),kins['svl'].max()

norm = plt.Normalize(min_,max_)

#each as a single graph
for i in np.arange(len(variables)):
    for j in np.arange(len(variables)):
        plt.figure()
        plt.scatter(B[variables[i]],B[variables[j]],c=B['svl'],cmap=cmap,marker='o',norm=norm,label = 'Chrysopelea')
        plt.xlabel(labels[i])
        plt.ylabel(labels[j])
        plt.colorbar().set_label('Snout-Vent Length (cm)',rotation=-270)
        plt.legend()
        
        axes = plt.gca()
        y_min, y_max = axes.get_ylim()
        x_min, x_max = axes.get_xlim()
        
        plt.savefig(cwd+"/Graphs/Contrasts/"+variables[i]+'_'+variables[j]+'_CP_gsrestrict.png')
        plt.close()

        plt.figure()
        plt.scatter(A[variables[i]],A[variables[j]],c=A['svl'],cmap=cmap,marker='D',norm=norm,label='Dendrealphis')
        plt.xlabel(labels[i])
        plt.ylabel(labels[j])
        plt.ylim((y_min,y_max))
        plt.xlim((x_min,x_max))
        plt.colorbar().set_label('Snout-Vent Length (cm)',rotation=-270)
        plt.legend()
        plt.savefig(cwd+"/Graphs/Contrasts/"+variables[i]+'_'+variables[j]+'_DP_gsrestrict.png')
        plt.close()

# # #as a grid, separate figures
# plt.rcParams["font.size"] = "8"

# fig,ax = plt.subplots(len(variables),len(variables))
# fig.suptitle("Chrysopelea")
# fig1,ax1 = plt.subplots(len(variables),len(variables))
# fig1.suptitle("Dendrelaphis")


# for i in np.arange(len(variables)):
#     for j in np.arange(len(variables)):
#         ax[i,j].scatter(B[variables[i]],B[variables[j]],c=B['svl'],cmap=cmap,marker='o',norm=norm,s=3)
#         y_min, y_max = ax[i,j].get_ylim()     
#         x_min, x_max = ax[i,j].get_xlim()
#         ax1[i,j].scatter(A[variables[i]],A[variables[j]],c=A['svl'],cmap=cmap1,marker='o',norm=norm,s=3)
#         ax1[i,j].set_ylim([y_min,y_max])
#         ax1[i,j].set_xlim([x_min,x_max])
        
#         ax[i,0].set_ylabel(ablabels[i])
#         ax[0,j].set_title(ablabels[j])
#         ax1[i,0].set_ylabel(ablabels[i])
#         ax1[0,j].set_title(ablabels[j])
        
#         ax[i,j].axes.xaxis.set_visible(False)
#         ax[i,j].set_yticklabels([]) 
#         ax[i,j].set_yticks([]) 
#         ax1[i,j].axes.xaxis.set_visible(False)
#         ax1[i,j].set_yticklabels([])
#         ax1[i,j].set_yticks([])

#as a grid, same figure
plt.rcParams["font.size"] = "8"

fig,ax = plt.subplots(len(variables),len(variables))
fig.suptitle("Chrysopelea and Dendrelaphis")


for i in np.arange(len(variables)):
    for j in np.arange(len(variables)):
        ax[i,j].scatter(B[variables[i]],B[variables[j]],c=B['svl'],cmap=cmap,marker='o',norm=norm,s=3)
        ax[i,j].scatter(A[variables[i]],A[variables[j]],c=A['svl'],cmap=cmap1,marker='o',norm=norm,s=3)
        
        ax[i,0].set_ylabel(ablabels[i])
        ax[0,j].set_title(ablabels[j])

        ax[i,j].axes.xaxis.set_visible(False)
        ax[i,j].set_yticklabels([]) 
        ax[i,j].set_yticks([]) 

### Comparisons

In [25]:
cpairs = []

for size,ind in zip(list(B['svl'].values),list(B['ID'].values)):
    if not (size,ind) in cpairs:
        cpairs.append((size,ind))    
cpairs.sort()

pairs = []
for i,j in zip(list(A['svl'].values),list(A['ID'].values)):
    if not (i,j) in pairs:
        if not j == "2019DP15" and not j == "2019DP13":
            pairs.append((i,j)) 
pairs.sort()

all_pairs = pairs+cpairs
len(all_pairs)

22

In [178]:
#special Z graph on same axes

a = 11
b = 2

i = 0
fig, ax = plt.subplots(a,b,sharey=True,sharex=True,figsize=(4,8),dpi=100)

for j in range(0,a):
    for k in range(0,b):
        n = b*j+k
        ID = all_pairs[n][1]
        svl = np.round(all_pairs[n][0],2)
        
        if n<15 and ID[-3] == 'C':
                shape = 'D'
                size = 10
        else:
            shape = 'o'
            size = 20

        C = kins[kins['ID']==ID]
        x = C[IVs[i]]
        y0 = C[DVs[6]]
        y1 = C[DVs[7]]
        
        ax[j,k].axhline(0,c='#808080',zorder=0)
        ax[j,k].scatter(x,y0,c='k',marker=shape,edgecolors='#808080',s=size,alpha=1,zorder=2)
        ax[j,k].scatter(x,y1,c='k',marker=shape,edgecolors='#808080',s=size,alpha=0.5,zorder=3)
        ax[j,k].plot((x,x),(y0, y1),'--',c='#D3D3D3',zorder=1)

        at = AnchoredText(str(svl)+" cm", prop=dict(fontweight='semibold',size=4),
                          frameon=True, loc='upper left')
        at.patch.set_boxstyle("round,pad=0.1,rounding_size=0.2")
        at.patch.set_edgecolor("k")
        at.patch.set_facecolor("#D3D3D3")
        at.patch.set_alpha(0.5)

        ax[j,k].add_artist(at)


plt.subplots_adjust(bottom = 0.2)
plt.figtext(0.475,0.14,I_labels[i])
ax[4,0].set_ylabel("Vertical Position (SVL)",fontsize=10)

legend_elements = [Line2D([0], [0], marker='o', color='k', label="AF",
                      markerfacecolor='k', markersize=10),
               Line2D([0], [0], marker='o', color='k', label='Max',
                      markerfacecolor='#808080', markeredgecolor = 'k',alpha=0.5, markersize=10)]
fig.legend(handles=legend_elements, loc='upper center',ncol=2)

# MISC past things
Need to put behavior columns back into spreadsheets if using these in the future. 

# Head Trajectories

In [108]:
cp_trials = [each['tn'] for each in all_imported]
NC_inds = [cp_trials.index(n) for n in list(cp_df['tn'].values)]

NC_data = [all_imported[n] for n in NC_inds]
NC_trials = [sm_pos[n] for n in NC_inds]
NC_vels = [sm_vel[n] for n in NC_inds]

In [176]:
#equations of motion for a projectile

def projectile(v, p, x, h,hz): 
    """creates a time series of position values in x (horizontal) and z (vertical)
    for a projectile launched with velocity v from initial position p until it 
    passes a particular goal position position (x,h); i.e. when the projectile's position is >x and <h.
    
    inputs: v (2x1 initial velocity vector, (vx,vz)) 
            p (2x1 initial position vector, (x0, z0)) 
            x (float, x criteria for stopping time series)
            h (float, z criterai for stopping time series)
    output: P, a 2xn array of position points."""
    
    t = float(1/hz)
    (x0, z0) = p
    (vx, vz) = v
    
    xt = x0
    zt = z0
    P = [[x0,z0]]
    v = vz
    V = [[vx,vz]]
    
    while zt>h or v>0:
        xt = vx*t + x0
        zt = z0 + vz*t - 0.5*980*t**2
        v = -980*t+vz
        P.append([xt,zt])
        V.append([vx,v])
        t = t+float(1/hz)
        
    
    P = np.array(P)
    V = np.array(V)
    return P,V

### Chrysopelea trajectories - scatter, broken out in dimensions

In [193]:
for n in np.arange(len(NC_data)):
    tn = NC_data[n]['tn']

    acc = cp_df['acceleration'][n]
    frames = list(NC_data[n]['fn'])
    acc = int(acc)
    acc_i = frames.index(acc)             

    snake = NC_trials[n][:,0,:]/10.0 #to cm, head marker data.
    peak_ = np.nanargmax(snake[acc_i:,2])+acc_i #find location of peak Z
    vel = np.array(NC_vels[n][:,0,:])/10.0 #to cm
    hz = NC_data[n]['fr']
    nte = NC_data[n]['nte']/10.0 #mm to cm

    p = [snake[peak_,0], snake[peak_,2]]
    v = [vel[peak_-1,0], vel[peak_-1,2]]
    P,V = projectile(v, p, nte[0], snake[-1,2], hz) #projectile traj ends at same height as snake head at landing, after passing end of target branch (in X)
    
    time1 = np.arange(len(snake[acc_i:]))
    tstart = time1[peak_-acc_i]
    time = np.arange(tstart,len(V)+tstart)

    time = time/hz
    time1 = time1/hz
    
    fig,ax = plt.subplots(4,sharex = True)

    #actual velocity
    ax[1].set_ylabel("Vz (cm/s)")
    ax[1].scatter(time1,vel[acc_i:,2],edgecolors = "k", facecolors="k",label="observed")
    ax[1].scatter(time,V[:,1],edgecolors = '#808080',facecolors='none',label="proj. from peak")

    ax[3].set_ylabel("Vx (cm/s)")
    ax[3].scatter(time1,vel[acc_i:,0],edgecolors = "k", facecolors="k",label="observed")
    ax[3].scatter(time,V[:,0],edgecolors = '#808080',facecolors='none',label="proj. from peak")

    #actual position
    ax[0].set_ylabel("Z (cm)")
    ax[0].axhline(nte[2]) #z target position
    ax[0].scatter(time1,snake[acc_i:,2],edgecolors = "k", facecolors="k",label="observed")
    ax[0].scatter(time,P[:,1],edgecolors = '#808080',facecolors='none',label="proj. from peak")

    ax[2].set_ylabel("X (cm)")
    ax[2].axhline(nte[0])
    ax[2].scatter(time1,snake[acc_i:,0],edgecolors = "k", facecolors="k",label="observed")
    ax[2].scatter(time,P[:,0],edgecolors = '#808080',facecolors='none',label="proj. from peak")

    ax[3].set_xlabel("Time (s)")
    plt.legend()

    ax[0].set_title('Trial '+str(tn)+", SVL: "+str(NC_data[n]['svl'])+", GS: "+str(np.round(NC_data[n]['gs_%'],decimals=1))+"% SVL")
    plt.savefig(cwd+'/Graphs/Landing control graphs/Trial '+str(tn)+"_landcontrol_zoom.pdf")

    plt.close()

In [180]:
plt.figure()
plt.scatter(P[:,0],P[:,1],c='#808080')
plt.scatter(snake[:,0],snake[:,2],c='k')

### Dendrelaphis trajectory breakouts - braking?

In [196]:
#import Dendrelaphis data
dp_df = pd.read_csv(cwd+'/Dendrelaphis NC Data.csv')
dp_df.dropna(axis=0,how="all",inplace=True)
dp_df.dropna(axis=1,how="all",inplace=True)
dp_df.head()

#create list of all head position data
pfiles = glob.glob(cwd+'/Processed Files/*')
pfiles.sort()
position = [pfiles[i] for i in np.arange(len(pfiles)) if pfiles[i][-12:-4]=='smoothed']
dp_pos = [pd.read_csv(item,header=None) for item in position]

#create list of all head velocity data
velocity = [pfiles[i] for i in np.arange(len(pfiles)) if pfiles[i][-12:-4]=='velocity']
dp_vel = [pd.read_csv(item,header=None) for item in velocity]

In [225]:
print(tn)
print(len(P),len(V),len(snake[peak_:]),len(vel[peak_:]))
print(len(time),len(time),len(time1),len(time1[1:-1]))

4 4 8 8
4 4 8 6


In [232]:
for n in np.arange(len(dp_df)):
    tn = dp_df['tn'][n]

    acc_i = dp_df['acceleration'][n]-dp_df['start'][n]
    snake = np.array(dp_pos[n])*100.0 #m to cm, head marker data.
    peak_ = np.argmax(snake[acc_i:,2])+acc_i #find location of peak Z
    vel = np.array(dp_vel[n])*100.0 #m to cm
    hz = 120 #120 fps on gp camera for all trials
    nte = np.array([dp_df['targ_x'][n],dp_df['targ_y'][n],dp_df['targ_z'][n]])*100.0 #m to cm

    p = [snake[peak_,0], snake[peak_,2]]
    v = [vel[peak_-1,0], vel[peak_-1,2]]
    P,V = projectile(v, p, nte[0], snake[-1,2], hz) 
    #projectile traj ends at same height as snake head at landing, after passing end of target branch (in X)
    
    time1 = np.arange(len(snake[acc_i:]))
    tstart = time1[peak_-acc_i]
    time = np.arange(tstart,len(V)+tstart)

    time = time/hz
    time1 = time1/hz
    
    if not tn == 93:
    
        fig,ax = plt.subplots(4,sharex = True)

        #actual velocity
        ax[1].set_ylabel("Vz (cm/s)")
        ax[1].scatter(time1[1:-1],vel[acc_i:,2],edgecolors = "k", facecolors="k",label="observed")
        ax[1].scatter(time,V[:,1],edgecolors = '#808080',facecolors='none',label="proj. from peak")

        ax[3].set_ylabel("Vx (cm/s)")
        ax[3].scatter(time1[1:-1],vel[acc_i:,0],edgecolors = "k", facecolors="k",label="observed")
        ax[3].scatter(time,V[:,0],edgecolors = '#808080',facecolors='none',label="proj. from peak")

        #actual position
        ax[0].set_ylabel("Z (cm)")
        ax[0].axhline(nte[2]) #z target position
        ax[0].scatter(time1,snake[acc_i:,2],edgecolors = "k", facecolors="k",label="observed")
        ax[0].scatter(time,P[:,1],edgecolors = '#808080',facecolors='none',label="proj. from peak")

        ax[2].set_ylabel("X (cm)")
        ax[2].axhline(nte[0])
        ax[2].scatter(time1,snake[acc_i:,0],edgecolors = "k", facecolors="k",label="observed")
        ax[2].scatter(time,P[:,0],edgecolors = '#808080',facecolors='none',label="proj. from peak")

        ax[3].set_xlabel("Time (s)")
        plt.legend()

        ax[0].set_title('Trial '+str(tn)+", SVL: "+str(NC_data[n]['svl'])+", GS: "+str(np.round(NC_data[n]['gs_%'],decimals=1))+"% SVL")
        plt.savefig(cwd+'/Graphs/Landing control graphs/DP_'+str(tn)+"_landcontrol_zoom.pdf")

        plt.close()

### Chrysopelea trajectories

In [ ]:
# graphing all head trajectories as separate PDF files

for n in np.arange(len(NC_trials)):
    plt.figure()
    
    trial = NC_data[n]['tn']
    snake = cp_heads[n]/1000.0 #to meters
    vel = np.array(NC_vels[n][:,0,:])/1000.0 #to meters
    hz = NC_data[n]['fr']
    nte = NC_data[n]['nte']/1000.0 #mm to meters
    
    #grabbing frame indices
    acc = cp_df['acceleration'][n] #acceleration frame
    stop = cp_df['stop'][n]  
    air = cp_df['airborne'][n]
    frames = list(NC_data[n]['fn']) #for translating frame into data index 
    stop_i = frames.index(stop)

    if not np.isnan(acc):
        acc = int(acc)
        acc_i = frames.index(acc) 
    
    else:
        acc_i = 0
        
    if not np.isnan(air):
        air = int(air)
        air_i = frames.index(air)
        
    else:
        air_i = stop_i-1
        
    #make positions relative to origin.
    if trial in [85,93,94]:
        oe1 = snake[0] #origin marker was missing for these trials
        
    else:
        oe1 = [0,0,0] #origin position
        
    snake = snake[:stop_i,:] - oe1 #make positions relative to origin.
    
    #fitting projectile trajectories
    Projs = []
    test_points = range(acc_i, air_i+1) #make projectile from each possible point from acc start to when becomes airborne.
    test_ranges = []
    #for snakes with no acc: starts from beginning of trial. 
    #for snakes with no air: ends at end of trial. 

    for each in test_points:
        p = [snake[each,0], snake[each,2]]
        m = len(snake) - each
        v = [vel[each-1,0], vel[each-1,2]]
        P = projectile(v, p, nte[0], snake[-1,2]) #projectile traj ends at same height as snake head at landing, after passing end of target branch (in X)
        if len(P) > 1:
            Projs.append(P)
            r = P[-1,0] - snake[acc_i,0] #final trajectory position - starting snake position, x distance between
            R = abs(r)
            test_ranges.append(R)
        
    R_max_i = np.argmax(test_ranges)
    ran = Projs[R_max_i]
    plt.plot(ran[:,0],ran[:,1],'o',color='#59b5f0',markersize=2,zorder=2,label ="Max Range")
    
    peak_ = np.nanargmax(snake[acc_i:,2])+acc_i
    plt.plot(snake[peak_,0],snake[peak_,2],'o',color='#87dafb',markersize=2,zorder=2,label ="Peak")
    plt.plot(snake[acc_i,0],snake[acc_i,2],'o', color='#c0fff6', markersize=2, label = 'Acc frame', zorder=2)
    plt.plot(snake[:,0],snake[:,2],'o', color='#91f3b7',markersize=2, label='head trajectory',zorder = 1)
    plt.plot(nte[0],nte[2],'o', color='#69e76d',markersize=2, label='target end',zorder = 1)

    if not np.isnan(air):      
        plt.plot(snake[air_i,0],snake[air_i,2],'o',color='#3ad44d',markersize=2,zorder=2,label ="Airborne frame")
        

    plt.gca().set_aspect('equal')
    handles,labels = plt.gca().get_legend_handles_labels()
    plt.gcf().legend(handles, labels, loc='upper center', ncol = 3)
    plt.title('Trial '+str(trial)+", SVL: "+str(NC_data[n]['svl'])+", Distance traveled: "+str(np.round(NC_data[n]['total_a'],decimals=1))+"% SVL")
    plt.savefig(cwd+'/Graphs/X-Z Trajectories - Smoothed/CP Head Trajectories/Trial '+str(trial)+"_headtrajs.pdf")

    plt.close()
    

In [ ]:
# graphing all COM trajectories as separate PDF files

#Step 1: generate COM trajectories

def fix_single_frame(SP_snake, spaces):
    """
    NOTE: must be at least 2 markers present in the frame.
    purpose: corrects an array of snake position data from a single frame, 
    and corresponding list of marker spacings, for markers that are missing 
    because they weren't visible in that frame
    
    snake = n x 3 array containing marker position data in 3 dimensions for n markers, measured in meters
    mark_spaces = list of n-1 spacings in meters between n markers
    """

    del_list = []
    n_snake = np.copy(SP_snake)
    n_sp = np.array(spaces)

    for t in np.arange(len(n_sp)+1):
        mark = n_snake[t,0]
        if np.isnan(mark): #make a list of markers to delete - delete ones with nans
            del_list.append(t)

    for l in reversed(del_list):
        n_snake=np.delete(n_snake,l,0) #delete the rows in the snake array
        if l==len(n_sp): #if the index corresponds to the last marker in the spacing array, just delete it
            n_sp=np.delete(n_sp,l-1)
        else:
            n_sp[l]=n_sp[l-1]+n_sp[l] #if the index corresponds to some other marker, get the appropriate spacing
            n_sp=np.delete(n_sp,l-1) #and then delete.

    
    return n_snake, n_sp

marks_s = [np.array(NC_data[i]['cms'])/100.0 for i in np.arange(len(NC_data))] #marker spacings are in cm; change to an array and divide by 100 to get meters.
NC_coms = []

for ind in np.arange(len(NC_trials)):
    snake = NC_trials[ind]/1000.0 #change mm to m
    svl = NC_data[ind]['svl']
    spaces = marks_s[ind][1:]
    mass_total = NC_data[ind]['mass']/1000.0 #change g to Kg
    
    #grabbing frame indices
    acc = cp_df['acceleration'][ind] #acceleration frame
    stop = cp_df['stop'][ind]  
    air = cp_df['airborne'][ind]
    frames = list(NC_data[ind]['fn']) #for translating frame into data index 
    stop_i = frames.index(stop)

    if not np.isnan(acc):
        acc = int(acc)
        acc_i = frames.index(acc) 
    
    else:
        acc_i = 0
        
    if not np.isnan(air):
        air = int(air)
        air_i = frames.index(air)
        
    else:
        air_i = stop_i-1
        
    snake = snake[acc_i:stop_i,:,:]
    com = np.empty((len(snake),3))

        
    for frame in np.arange(len(snake)):
    
        ### Section 1: Create a spline across the markers in the frame.
        SP_snake = np.copy(snake[frame,:,:])
        sp = spaces
        n_snake,n_sp = fix_single_frame(SP_snake,sp) #correct for any missing markers in the frame.
    
        #now make the spline
        out = global_natural_spline(n_snake,np.array(n_sp),1000) #fit a spline
        r, dr, ddr, dddr, ts, ss, seg_lens, lengths_total, idx_pts = out    
        
        ### Section 2: Find the COM
        #add density to the spline
        density_df = pd.read_csv(cwd+'/snake_density.csv', index_col=0)
        s_rho, body_rho = density_df.values.T
        density_body = np.interp(ts / svl, s_rho, body_rho)
        mass_body = mass_total * density_body / density_body.sum()
        
        for i in np.arange(3):
            com_i = sum(r[:,i]*mass_body)/mass_total
            com[frame,i] = com_i
        
    NC_coms.append(com)

In [ ]:
with open('NC_com_data.pkl', 'wb') as var:
    dill.dump(NC_coms,var)
    

In [ ]:
#check if there are nans
for each in NC_coms:
    if (each==np.isnan).any():
        
        print("has nans")
        

In [ ]:
#graphs of COM Motion compares to head motion

for ind in np.arange(len(NC_trials)):
    trial = NC_data[ind]['tn']
    frames = list(NC_data[ind]['fn']) #for translating frame into data index 
    acc_i = 0
    air_i = len(frames)-1
    
    if not(np.isnan(cp_df['acceleration'][ind])):
        acc = cp_df['acceleration'][ind] #acceleration frame
        acc_i = frames.index(int(acc)) 
    
    if not(np.isnan(cp_df['airborne'][ind])):
        air = cp_df['airborne'][ind]
        air_i = frames.index(int(air))
        
           
    plt.figure()
    plt.plot(NC_coms[ind][:,0],NC_coms[ind][:,2],'o', label = 'COM')
    plt.plot(NC_trials[ind][acc_i:,0,0]/1000,NC_trials[ind][acc_i:,0,2]/1000,'o', label = 'Head')
    plt.plot(NC_trials[ind][air_i,0,0]/1000,NC_trials[ind][air_i,0,2]/1000, 'o', label = 'Airborne')
    plt.title(cp_df['tn'][ind])
    plt.legend()
    plt.savefig('/Users/grahmich/Desktop/Jupyter Notebooks/Graphs/X-Z Trajectories - Smoothed/CP Head Trajectories/Trial '+str(trial)+"_COMtrajs.pdf")
    plt.close()

## Load Chrysopelea data

In [2]:
#import chrysopelea data
import dill

with open('smooth_pos_v8.pkl', 'rb') as var:
    sm_pos = dill.load(var)
    
with open('smooth_vel_v8.pkl', 'rb') as var:
    sm_vel = dill.load(var)
    
with open('raw_dictionaries_v9.pkl', 'rb') as var:
    all_imported = dill.load(var)
    
cp_df = pd.read_csv(cwd+'/Chrysopelea NC Data.csv')

cp_trials = [each['tn'] for each in all_imported]
NC_inds = [cp_trials.index(n) for n in list(cp_df['tn'].values)]
NC_data = [all_imported[n] for n in NC_inds]

## Load Dendrelaphis data

In [3]:
#create list of all head position data
pfiles = glob.glob(cwd+'/Processed Files/*')
pfiles.sort()
position = [pfiles[i] for i in np.arange(len(pfiles)) if pfiles[i][-12:-4]=='smoothed']

In [4]:
kins = pd.read_csv(cwd+'/all_data_for_stats.csv')
loops = list(cp_df['loop_af'].values)

A = kins[kins['genus']=="Dendrelaphis"]
B = kins[kins['genus'] == "Chrysopelea"]
dmx = max(A['gscm_rel'])

In [5]:
#Load dendrelaphis data
dp_df = pd.read_csv(cwd+'/Dendrelaphis NC Data.csv')
dp_df.dropna(axis=0,how="all",inplace=True)
dp_df.dropna(axis=1,how="all",inplace=True)
dp_df.head()

,tn,ID,svl,type,gs_att,start,transition,acceleration,stop,reference,at_tf,at_af,loop_af,loop_af_sp,orig_x,targ_x,orig_y,targ_y,orig_z,targ_z
0,9,2018DP01,113.990000,inline,50.0,1,1401.0,1401,1487,1498,No,No,0,0.0,-0.008115,0.419087,-0.293575,-0.293575,-0.312613,-0.538487
1,10,2018DP01,113.990000,inline,46.0,1,865.0,865,918,1,No,No,0,0.0,-0.010670,0.378284,-0.316885,-0.316885,-0.295888,-0.477425
2,11,2018DP01,113.990000,inline,45.2,1,365.0,365,443,1,No,No,0,0.0,-0.016344,0.390685,-0.315580,-0.315580,-0.262483,-0.464088
3,14,2018DP01,113.990000,inline,55.0,1,1494.0,1494,1585,847,No,No,0,0.0,0.024407,0.452703,-0.280117,-0.280117,-0.365285,-0.557537
4,42,2018DP02,127.443333,inline,53.0,8063,8712.0,8712,8757,4468,No,No,0,0.0,-0.342431,0.202844,-0.399090,-0.399090,0.091379,0.114363


## Chrysopelea: Graph of loops/head position at AF

In [16]:
#(1) does increasing gap size (presented) lead to increasing use of below-branch acceleration starts?
#(2) does increasing gap size (presented) lead to increasing use of loops?
loops = list(cp_df['loop_af'].values)
norm = mp.colors.Normalize(vmin=0, vmax=5) #snake IDs
cmap = mp.cm.get_cmap('autumn')
IDs = [85,88,89,90,94,95]

plt.figure()

for i in np.arange(len(loops)):

    gs = NC_data[i]['gs_%']
    
    if gs < dmx:
        zp = B['zpaf_rel'][i]
        ID = int(B['ID'][i])
        color = cmap(norm(IDs.index(ID)))

        if loops[i] == 1:
            plt.scatter(gs,zp,color=color)

        elif loops[i] == 0:
            plt.scatter(gs,zp,facecolors='none',edgecolor=color)

        else:
            print("error: entry is not 1 or 0")
        
plt.ylim(min(kins['zpaf_rel'])-0.05,max(kins['zpaf_rel'])+0.05)
plt.xlim(min(kins['gscm_rel'])-5,max(kins['gscm_rel'])+5)
       
plt.xlabel("Presented gap size (%SVL)")
plt.ylabel("Z - Head position (%SVL)")
plt.title("Chrysopelea begin acceleration below the branch when presented with larger gaps, and they usually use loops")

legend_elements = [Line2D([0], [0], marker='o', color='#FFFFFF', label="Loop",
                          markerfacecolor='#808080', markersize=10),
                   Line2D([0], [0], marker='o', color='#FFFFFF', label='No loop',
                          markerfacecolor='none', markeredgecolor = '#808080', markersize=10),
                  Line2D([0], [0], marker='o', color='#FFFFFF', label="Snake 85",
                          markerfacecolor=cmap(norm(0)), markersize=8),
                   Line2D([0], [0], marker='o', color='#FFFFFF', label='Snake 88',
                          markerfacecolor=cmap(norm(1)), markersize=8),
                 Line2D([0], [0], marker='o', color='#FFFFFF', label='Snake 89',
                          markerfacecolor=cmap(norm(2)), markersize=8),
                    Line2D([0], [0], marker='o', color='#FFFFFF', label='Snake 90',
                          markerfacecolor=cmap(norm(3)), markersize=8),
                  Line2D([0], [0], marker='o', color='#FFFFFF', label='Snake 94',
                          markerfacecolor=cmap(norm(4)), markersize=8),
                   Line2D([0], [0], marker='o', color='#FFFFFF', label='Snake 95',
                          markerfacecolor=cmap(norm(5)), markersize=8)]

plt.axhline(0,c="#D3D3D3")
plt.legend(handles=legend_elements, ncol=4)

In [ ]:
norm = mp.colors.Normalize(vmin=0, vmax=5) #snake IDs
cmap = mp.cm.get_cmap('autumn')
IDs = [85,88,89,90,94,95]


fig, ax = plt.subplots()
during = [each['total_a'] for each in NC_data] #total distance traveled from acc start to landing

for i in np.arange(len(loops)):
    zp = NC_data[i]['head_a']
    dist = NC_data[i]['total_a']
    ID = cp_df['ID'][i]
    color = cmap(norm(IDs.index(ID)))
    
    if loops[i] == "Yes":
        ax.scatter(zp,dist,color=color)
        
    elif loops[i] == "No":
        ax.scatter(zp,dist,facecolors='none',edgecolor=color)
        
    else:
        print("error: entry is not yes or no")
        
        
ax.set_xlabel("Starting head height (%SVL)")
ax.set_ylabel("Distance traveled (%SVL)")


legend_elements = [Line2D([0], [0], marker='o', color='#FFFFFF', label="Loop",
                          markerfacecolor='#808080', markersize=10),
                   Line2D([0], [0], marker='o', color='#FFFFFF', label='No loop',
                          markerfacecolor='none', markeredgecolor = '#808080', markersize=10),
                  Line2D([0], [0], marker='o', color='#FFFFFF', label="Snake 85",
                          markerfacecolor=cmap(norm(0)), markersize=8),
                   Line2D([0], [0], marker='o', color='#FFFFFF', label='Snake 88',
                          markerfacecolor=cmap(norm(1)), markersize=8),
                 Line2D([0], [0], marker='o', color='#FFFFFF', label='Snake 89',
                          markerfacecolor=cmap(norm(2)), markersize=8),
                    Line2D([0], [0], marker='o', color='#FFFFFF', label='Snake 90',
                          markerfacecolor=cmap(norm(3)), markersize=8),
                  Line2D([0], [0], marker='o', color='#FFFFFF', label='Snake 94',
                          markerfacecolor=cmap(norm(4)), markersize=8),
                   Line2D([0], [0], marker='o', color='#FFFFFF', label='Snake 95',
                          markerfacecolor=cmap(norm(5)), markersize=8)]



ax.legend(handles=legend_elements, loc='upper right')
ax.axvline(0)

KeyError: 'total_a'

## Dendrelaphis: Graphs of loops/head position and 2

### Graph 1

In [17]:
#(1) does increasing gap size (presented) lead to increasing use of below-branch acceleration starts?
#(2) does increasing gap size (presented) lead to increasing use of loops?
loops = list(dp_df['loop_af'].values)
norm = mp.colors.Normalize(vmin=0, vmax=19) #snake IDs
cmap = mp.cm.get_cmap('winter')

L = list(np.unique(np.array(dp_df['ID'].values)))
P = [string[-2:] for string in L]
Q = sorted(zip(P,L))
IDs = [j for i,j in Q]

for i in np.arange(len(A)):
    trial = A['tn'].iloc[i]
    gs = A['gscm_rel'].iloc[i]
    zp = A['zpaf_rel'].iloc[i]
    ID = A['ID'].iloc[i]
    color = cmap(norm(IDs.index(ID)))
    
    if loops[i] != 0:
        plt.scatter(gs,zp,color=color)
        
    else:
        plt.scatter(gs,zp,facecolors='none',edgecolor=color)
        
        
plt.xlabel("Presented gap size (%SVL)")
plt.ylabel("Z - Head position (%SVL)")
plt.title("Dendrelaphis do not alter head position with gap size, and they don't typically use loops")

legend_elements = [Line2D([0], [0], marker='o', color='#FFFFFF', label="Loop",
                          markerfacecolor='#808080', markersize=10),
                   Line2D([0], [0], marker='o', color='#FFFFFF', label='No loop',
                          markerfacecolor='none', markeredgecolor = '#808080', markersize=10)]
for i in np.arange(len(IDs)):
    legend_elements.append(Line2D([0], [0], marker='o', color='#FFFFFF', label=IDs[i],
                          markerfacecolor=cmap(norm(i)), markersize=8))

plt.ylim(min(kins['zpaf_rel'])-0.05,max(kins['zpaf_rel'])+0.05)
plt.xlim(min(kins['gscm_rel'])-5,max(kins['gscm_rel'])+5)
       
plt.axhline(0,c="#D3D3D3")
plt.legend(handles=legend_elements,ncol=5)

In [ ]:
#graph 1 by individual
loops = list(dp_df['loop_af'].values)
norm = mp.colors.Normalize(vmin=0, vmax=19) #snake IDs
cmap = mp.cm.get_cmap('winter')

L = list(np.unique(np.array(dp_df['ID'].values)))
P = [string[-2:] for string in L]
Q = sorted(zip(P,L))
IDs = [j for i,j in Q]
IDs = IDs[2:] #no data for snake 1 or 2 (yet?)

fig,ax = plt.subplots(int(len(IDs)/2),2,sharex=True,sharey=True)

for i in np.arange(len(position)):
    trial = int(position[i][-25:-22])
    gs = gs_svl[i]
    zp = head_a[i]
    
    ind = df_list.index(trial)
    ID = dp_df['ID'][ind]

    if IDs.index(ID) < int(len(IDs)/2):
        n = IDs.index(ID)
        j = 0
    else:
        n = IDs.index(ID)-int(len(IDs)/2)
        j = 1
    
    color = cmap(norm(IDs.index(ID)))
    
    if loops[ind] != "No":
        ax[n,j].scatter(gs,zp,color=color)
        
    else:
        ax[n,j].scatter(gs,zp,facecolors='none',edgecolor=color)
        
        
ax[int(len(IDs)/2)-1,0].set_xlabel("Presented gap size (%SVL)")
ax[int(len(IDs)/2)-1,1].set_xlabel("Presented gap size (%SVL)")
ax[int(len(IDs)/4),0].set_ylabel("Z - Head position (%SVL)")

# legend_elements = [Line2D([0], [0], marker='o', color='#FFFFFF', label="Loop",
#                           markerfacecolor='#808080', markersize=10),
#                    Line2D([0], [0], marker='o', color='#FFFFFF', label='No loop',
#                           markerfacecolor='none', markeredgecolor = '#808080', markersize=10)]
# for i in np.arange(len(IDs)):
#     legend_elements.append(Line2D([0], [0], marker='o', color='#FFFFFF', label=IDs[i],
#                           markerfacecolor=cmap(norm(i)), markersize=8))



# plt.legend(handles=legend_elements,ncol=10,loc='upper center')

### Graph 2

In [ ]:
#(3) is the use of loops associated with increased acceleration distance available?
#(4) is the use of loops associated with increased total distance traveled?
#(5) are below branch starts associated with increased acceleration distance available?
#(6) are below branch starts associated with increased total distance traveled?
norm = mp.colors.Normalize(vmin=0, vmax=19) #snake IDs
cmap = mp.cm.get_cmap('winter')

L = list(np.unique(np.array(dp_df['ID'].values)))
P = [string[-2:] for string in L]
Q = sorted(zip(P,L))
IDs = [j for i,j in Q]

fig, ax = plt.subplots()

for i in np.arange(len(position)):
    trial = int(position[i][-25:-22])
    zp = head_a[i]
    dist = total_a[i]
    
    ind = df_list.index(trial)
    ID = dp_df['ID'][ind]
    color = cmap(norm(IDs.index(ID)))
    
    if loops[ind] != "No":
        ax.scatter(zp,dist,color=color)
        
    else:
        ax.scatter(zp,dist,facecolors='none',edgecolor=color)
        
        
ax.set_xlabel("Starting head height (%SVL)")
ax.set_ylabel("Distance traveled (%SVL)")


# fig.suptitle("Going lower --> greater distance traveled for loops, not non-loops")
legend_elements = [Line2D([0], [0], marker='o', color='#FFFFFF', label="Loop",
                          markerfacecolor='#808080', markersize=10),
                   Line2D([0], [0], marker='o', color='#FFFFFF', label='No loop',
                          markerfacecolor='none', markeredgecolor = '#808080', markersize=10)]
for i in np.arange(len(IDs)):
    legend_elements.append(Line2D([0], [0], marker='o', color='#FFFFFF', label=IDs[i],
                          markerfacecolor=cmap(norm(i)), markersize=8))


ax.legend(handles=legend_elements, loc='upper center',ncol=10)
ax.axvline(0)

In [ ]:
# graph 2 by individual

loops = list(dp_df['loop_af'].values)
norm = mp.colors.Normalize(vmin=0, vmax=19) #snake IDs
cmap = mp.cm.get_cmap('winter')

L = list(np.unique(np.array(dp_df['ID'].values)))
P = [string[-2:] for string in L]
Q = sorted(zip(P,L))
IDs = [j for i,j in Q]
IDs = IDs[2:] #no data for snake 1 or 2 (yet?)

fig,ax = plt.subplots(int(len(IDs)/2),2,sharex=True,sharey=True)

for i in np.arange(len(position)):
    trial = int(position[i][-25:-22])
    zp = head_a[i]
    dist = total_a[i]
    
    ind = df_list.index(trial)
    ID = dp_df['ID'][ind]

    if IDs.index(ID) < int(len(IDs)/2):
        n = IDs.index(ID)
        j = 0
    else:
        n = IDs.index(ID)-int(len(IDs)/2)
        j = 1
    
    color = cmap(norm(IDs.index(ID)))
    
    if loops[ind] != "No":
        ax[n,j].scatter(zp,dist,color=color)
        
    else:
        ax[n,j].scatter(zp,dist,facecolors='none',edgecolor=color)
        
        
ax[int(len(IDs)/2)-1,0].set_xlabel("Starting head position")
ax[int(len(IDs)/2)-1,1].set_xlabel("Startng head position")
ax[int(len(IDs)/4),0].set_ylabel("Distance traveled")

## Both Genera: Graphs of loops/head position and loops vs af distance

### Graph 1

In [ ]:
loops = list(cp_df['loop_af'].values)
norm1 = mp.colors.Normalize(vmin=0, vmax=5) #snake IDs
cmap1 = mp.cm.get_cmap('autumn')
IDs = [85,88,89,90,94,95]

plt.figure()

for i in np.arange(len(loops)):
    gs = NC_data[i]['gs_%']
    zp = NC_data[i]['head_a']
    ID = cp_df['ID'][i]
    color = cmap1(norm1(IDs.index(ID)))
    
    if loops[i] == "Yes":
        plt.scatter(gs,zp,color=color)
        
    elif loops[i] == "No":
        plt.scatter(gs,zp,facecolors='none',edgecolor=color)
        
    else:
        print("error: entry is not yes or no")
        
    
loops = list(dp_df['loop_af'].values)
norm = mp.colors.Normalize(vmin=0, vmax=19) #snake IDs
cmap = mp.cm.get_cmap('winter')

L = list(np.unique(np.array(dp_df['ID'].values)))
P = [string[-2:] for string in L]
Q = sorted(zip(P,L))
IDs = [j for i,j in Q]


for i in np.arange(len(position)):
    trial = int(position[i][-25:-22])
    gs = gs_svl[i]
    zp = head_a[i]
    
    ind = df_list.index(trial)
    ID = dp_df['ID'][ind]
    color = cmap(norm(IDs.index(ID)))
    
    if loops[ind] != "No":
        plt.scatter(gs,zp,color=color)
        
    else:
        plt.scatter(gs,zp,facecolors='none',edgecolor=color)
        


plt.xlabel("Presented gap size (%SVL)")
plt.ylabel("Z - Head position (%SVL)")

legend_elements = [Line2D([0], [0], marker='o', color='#FFFFFF', label="Loop",
                          markerfacecolor='#808080', markersize=10),
                   Line2D([0], [0], marker='o', color='#FFFFFF', label='No loop',
                          markerfacecolor='none', markeredgecolor = '#808080', markersize=10),
                  Line2D([0], [0], marker='o', color='#FFFFFF', label="Snake 85",
                          markerfacecolor=cmap1(norm1(0)), markersize=8),
                   Line2D([0], [0], marker='o', color='#FFFFFF', label='Snake 88',
                          markerfacecolor=cmap1(norm1(1)), markersize=8),
                 Line2D([0], [0], marker='o', color='#FFFFFF', label='Snake 89',
                          markerfacecolor=cmap1(norm1(2)), markersize=8),
                    Line2D([0], [0], marker='o', color='#FFFFFF', label='Snake 90',
                          markerfacecolor=cmap1(norm1(3)), markersize=8),
                  Line2D([0], [0], marker='o', color='#FFFFFF', label='Snake 94',
                          markerfacecolor=cmap1(norm1(4)), markersize=8),
                   Line2D([0], [0], marker='o', color='#FFFFFF', label='Snake 95',
                          markerfacecolor=cmap1(norm1(5)), markersize=8)]


for i in np.arange(len(IDs)):
    legend_elements.append(Line2D([0], [0], marker='o', color='#FFFFFF', label=IDs[i],
                          markerfacecolor=cmap(norm(i)), markersize=8))



plt.legend(handles=legend_elements,ncol=2,loc='upper right')

### Graph 2

In [ ]:
#(3) is the use of loops associated with increased acceleration distance available?
#(4) is the use of loops associated with increased total distance traveled?
#(5) are below branch starts associated with increased acceleration distance available?
#(6) are below branch starts associated with increased total distance traveled?

#Chrysopelea
loops = list(cp_df['loop_af'].values)
norm1 = mp.colors.Normalize(vmin=0, vmax=5) #snake IDs
cmap1 = mp.cm.get_cmap('autumn')
IDs = [85,88,89,90,94,95]


fig, ax = plt.subplots()
during = [each['total_a'] for each in NC_data] #total distance traveled from acc start to landing

for i in np.arange(len(loops)):
    zp = NC_data[i]['head_a']
    dist = NC_data[i]['total_a']
    ID = cp_df['ID'][i]
    color = cmap1(norm1(IDs.index(ID)))
    
    if loops[i] == "Yes":
        ax.scatter(zp,dist,color=color)
        
    elif loops[i] == "No":
        ax.scatter(zp,dist,facecolors='none',edgecolor=color)
        
    else:
        print("error: entry is not yes or no")
        
#Dendrelaphis
loops = list(dp_df['loop_af'].values)
norm = mp.colors.Normalize(vmin=0, vmax=19) #snake IDs
cmap = mp.cm.get_cmap('winter')

L = list(np.unique(np.array(dp_df['ID'].values)))
P = [string[-2:] for string in L]
Q = sorted(zip(P,L))
IDs = [j for i,j in Q]

for i in np.arange(len(position)):
    trial = int(position[i][-25:-22])
    zp = head_a[i]
    dist = total_a[i]
    
    ind = df_list.index(trial)
    ID = dp_df['ID'][ind]
    color = cmap(norm(IDs.index(ID)))
    
    if loops[ind] != "No":
        ax.scatter(zp,dist,color=color)
        
    else:
        ax.scatter(zp,dist,facecolors='none',edgecolor=color)
        
        
ax.set_xlabel("Starting head height (%SVL)")
ax.set_ylabel("Distance traveled (%SVL)")
ax.legend(handles=legend_elements, loc='upper right')
ax.axvline(0)

legend_elements = [Line2D([0], [0], marker='o', color='#FFFFFF', label="Loop",
                          markerfacecolor='#808080', markersize=10),
                   Line2D([0], [0], marker='o', color='#FFFFFF', label='No loop',
                          markerfacecolor='none', markeredgecolor = '#808080', markersize=10),
                  Line2D([0], [0], marker='o', color='#FFFFFF', label="Snake 85",
                          markerfacecolor=cmap1(norm1(0)), markersize=8),
                   Line2D([0], [0], marker='o', color='#FFFFFF', label='Snake 88',
                          markerfacecolor=cmap1(norm1(1)), markersize=8),
                 Line2D([0], [0], marker='o', color='#FFFFFF', label='Snake 89',
                          markerfacecolor=cmap1(norm1(2)), markersize=8),
                    Line2D([0], [0], marker='o', color='#FFFFFF', label='Snake 90',
                          markerfacecolor=cmap1(norm1(3)), markersize=8),
                  Line2D([0], [0], marker='o', color='#FFFFFF', label='Snake 94',
                          markerfacecolor=cmap1(norm1(4)), markersize=8),
                   Line2D([0], [0], marker='o', color='#FFFFFF', label='Snake 95',
                          markerfacecolor=cmap1(norm1(5)), markersize=8)]


for i in np.arange(len(IDs)):
    legend_elements.append(Line2D([0], [0], marker='o', color='#FFFFFF', label=IDs[i],
                          markerfacecolor=cmap(norm(i)), markersize=8))

ax.legend(handles=legend_elements, loc='upper right',ncol=2)
ax.axvline(0)

# Velocities: Evidence of Braking?

In [37]:
#landing velocities: just NC trials

land_vs = np.empty((len(cp_df),3))

r = 0
for n in np.arange(len(sm_pos)):
    test_snake = sm_pos[n][:,0,:]

    if all_imported[n]['tn'] in list(cp_df['tn'].values):
        fr = all_imported[n]['fr']
        t_vals = [each/fr for each in np.arange(np.shape(test_snake)[0])]
        i = int(fr*0.07)    
        lv = np.empty(3)

        for d in [0,1,2]: #linear fit to each dimension for the final 0.07s
            res_snake = test_snake[:,d] 
            v_line, residuals, rank, singular_values, rcond = np.polyfit(t_vals[-i:],res_snake[-i:],1,full=True)
            vel_pts = [v_line[1]+v_line[0]*each for each in t_vals]

            lv[d]=(v_line[0]/1000.0)
            
        land_vs[r,:] = lv
        r = r+1

In [82]:
#compare head Vx, Vy, and Vz at peak of lunge and at landing.
af_v = np.empty((len(cp_df),3))
max_v = np.empty((len(cp_df),3))

r = 0
for i in np.arange(len(sm_vel)):
    if all_imported[i]['tn'] in list(cp_df['tn'].values):          
        frame = cp_df['acceleration'][r]
        f_ind = np.where(all_imported[i]['fn'] == frame)[0][0]
        af_velo = sm_vel[i][f_ind,0,:]/1000.0 #change from mm/s to m/s
        af_v[r,:]=af_velo
                 
        resultant = np.linalg.norm(sm_vel[i][:,0,:],axis=1)
        maxInd = np.nanargmax(resultant)
        # print(maxInd,resultant[maxInd])
        max_v[r,:] = sm_vel[i][maxInd,0,:]/1000.0 #change from mm/s to m/s
        r = r+1

In [103]:
# graph
fig,ax = plt.subplots(3)

min_,max_ = B['ID'].min(),B['ID'].max()
norm = plt.Normalize(min_,max_)

labels = ["X","Y","Z"]

ax[0].set_title("Landing vs Max velocities")
ax[2].set_xlabel("Gap Size %SVL")
for i in np.arange(3):
    ax[i].set_ylabel(labels[i])
    
    for j in np.arange(len(B['ID'])):
        ax[i].plot(B['gscm_rel'][j],max_v[j,i],c=cmap(norm(int(B['ID'][j]))),marker='D')
        ax[i].plot(B['gscm_rel'][j],land_vs[j,i],mfc='none',mec=cmap(norm(int(B['ID'][j]))),marker='o')

    
cmap = mp.cm.get_cmap('winter')

                     
legend_elements = [ Line2D([0], [0], marker='D', color='#FFFFFF', label="Max Resultant", markerfacecolor='#808080', markersize=10),
                    Line2D([0], [0], marker='o', color='#FFFFFF', label='Landing', markerfacecolor='none', markeredgecolor = '#808080', markersize=10),
                    Line2D([0], [0], marker='o', color='#FFFFFF', label='Snake 85',markerfacecolor=cmap(norm(85)), markersize=8),
                    Line2D([0], [0], marker='o', color='#FFFFFF', label='Snake 88',markerfacecolor=cmap(norm(88)), markersize=8),
                    Line2D([0], [0], marker='o', color='#FFFFFF', label='Snake 89',markerfacecolor=cmap(norm(89)), markersize=8),
                    Line2D([0], [0], marker='o', color='#FFFFFF', label='Snake 90',markerfacecolor=cmap(norm(90)), markersize=8),
                    Line2D([0], [0], marker='o', color='#FFFFFF', label='Snake 94',markerfacecolor=cmap(norm(94)), markersize=8),
                    Line2D([0], [0], marker='o', color='#FFFFFF', label='Snake 95',markerfacecolor=cmap(norm(95)), markersize=8)]
fig.legend(handles=legend_elements,ncol=4)

In [107]:
fig,ax = plt.subplots(3)

min_,max_ = B['ID'].min(),B['ID'].max()
norm = plt.Normalize(min_,max_)

labels = ["X","Y","Z"]

ax[0].set_title("Max Res V - Land V (positive = braking?)")
ax[2].set_xlabel("Gap Size %SVL")
for i in np.arange(3):
    ax[i].set_ylabel(labels[i])
    ax[i].axhline(0,c="#808080")
    
    for j in np.arange(len(B['ID'])):
        val = max_v[j,i] - land_vs[j,i]
        ax[i].plot(B['gscm_rel'][j],val,c=cmap(norm(int(B['ID'][j]))),marker='o')
